# Introduction  
### The following code was tested on:  
Python 3+ running on...  

ULB Cluster - user epb123

Please note - the code runs slowly due to the number of .count() we use for summary purposes. To arrive at a clean dataset quicker, all .count() in string outputs can be commented out. I would also recommend leaving it running overnight.

Additionally, our yellow data file is large - if you find yourself getting memory errors, it may be worth running FHV, FHVHV, and Green in one notebook, and Yellow separately in another. I have increased spark's memory allocation below, but on a personal computer it may throw an exception if the allocated memory is greater than the physical.

# Data cleaning

Now that the files in each sub-dataset conform to the same schema it becomes possible to check
the files for errors. To this end, for each sub-dataset:  
- First analyze what the valid values for each column are, based on the data dictionary available at the TLC website. (E.g., trip_distance should be a floating point value, and cannot be negative). Be sure to discuss your conclusions in your report.
- Write code that can be used to check all the validity constraints for this sub-dataset.
- For each file in the sub-dataset, detect the dirty records by running your validity constraints.
- Inspect the dirty records. Are there records that can be repaired ? Repairing is possible, for example, when the record has data of the incorrect type in the column, but it is clear how to convert that data to the desired type.
- Split the file into two: one containing the clean and repaired records, and one containing the dirty (and unrepairable) records. During the analysis of Section 2.4, we only use the clean and repaired records, the others are discarded.
- In your report, include a discussion that summarizes, for each file, the kinds of errors found (with statistics), how you repaired those errors (if applicable), and how many records are discarded (because they are dirty and unrepairable).


Here we import various packages to read and analyse our cleaned files

In [1]:
# Various imports
import os 
import glob
import math
import pickle
import ntpath
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
# Recover directories and filepaths

v_direc = pickle.load(open("v_direc",'rb'))
data_directory = pickle.load(open(v_direc + "data_folder",'rb'))
save_directory = pickle.load(open(v_direc + "save_directory",'rb'))

# This should not be set to true - see comment below
cluster = False

if cluster == True:
    cluster = False
    # Just in case - wouldn't want an error immediately, but also would like the future functionality.

Because of the read/write permissions issues in HDFS on the cluster, there is no way to run 2.3 in cluster mode without re-calculating the integrated datasets in this notebook. We therefore start spark in local mode, which will also work on the cluster (and is much faster than my laptop).

The below code will start Spark in local mode, using all available CPU cores.

We are setting spark.driver.memory to 16g here - the Yellow datasets consume a lot of memory with our memory inefficient code.
If you find that you are getting spark errors, you may have <16g memory on your computer.

In [3]:
# # -------------------------------
# # Start Spark in LOCAL mode
# # -------------------------------

if cluster == False:

    #This is needed to start a Spark session from the notebook
    os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=16g  pyspark-shell"

    from pyspark.sql import SparkSession

    # Stop any previously running spark session
    try: 
        spark
        print("Spark application already started. Terminating existing application and starting new one")
        spark.stop()
    except: 
        pass

    # Create a new spark session (note, the * indicates to use all available CPU cores)
    spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("H600Project") \
        .getOrCreate()

    #When dealing with RDDs, we work the sparkContext object. 
    #See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
    sc=spark.sparkContext

    from pyspark.sql.types import *
    from pyspark.sql.functions import col,udf,lit,struct
    from pyspark import SQLContext

    sql_sc = SQLContext(sc)

    print(sc.version)

2.4.4


The below code will start Spark on the ULB cluster.

In [4]:
# ----------------------------------------------------------------
# Start Spark on the ULB cluster ,with YARN as resource manager
# ----------------------------------------------------------------

if cluster == True:

    #This is needed to start a Spark session from the notebook
    os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=16g  pyspark-shell"

    from pyspark.sql import SparkSession

    # We need to set the following environment variable, so that Spark knows where YARN runs
    os.environ['HADOOP_CONF_DIR']="/etc/hadoop/conf"

    # Since we are accessing spark through it's python API, we need to make sure that all executor
    # instances run the same version of python. 
    # (and we want Anaconda to be used, so we have access to numpy, pandas, and so forth)
    # You will likely need to adjust this path if your run on a different cluster
    os.environ['PYSPARK_PYTHON']="/usr/local/anaconda3/bin/python"
    os.environ['PYSPARK_DRIVER_PYTHON']="/usr/local/anaconda3/bin/python"

    #The following lines are just there to allow this cell to be re-executed multiple times:
    #if a spark session was already started, we stop it before starting a new one
    #(there can be only one spark context per jupyter notebook)
    try: 
        spark
        print("Spark application already started. Terminating existing application and starting new one")
        spark.stop()
    except: 
        pass

    spark = SparkSession \
        .builder \
        .master("yarn") \
        .config("spark.executor.instances","4") \
        .appName("H600Project") \
        .getOrCreate()

    #When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
    sc=spark.sparkContext

    from pyspark.sql.types import *
    from pyspark.sql.functions import col,udf,lit,struct
    from pyspark.sql import SQLContext
    sql_sc = SQLContext(sc)

    print(sc.version)

In [5]:
import pyspark.sql.functions as sql

In [6]:
# Check that spark context is working, print its configuration
sc._conf.getAll()

[('spark.driver.port', '44289'),
 ('spark.app.id', 'local-1611630047402'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'H600Project'),
 ('spark.driver.host', 'publiclogin.hpda.ulb.ac.be'),
 ('spark.driver.memory', '16g'),
 ('spark.ui.showConsoleProgress', 'true')]

In [7]:
# Retrieve stored variables from memory

fhv_integratedPaths = pickle.load(open(v_direc + "fhv_integratedPaths",'rb'))
fhv_folderPath = pickle.load(open(v_direc + "fhv_folderPath",'rb'))

fhvhv_integratedPaths = pickle.load(open(v_direc + "fhvhv_integratedPaths",'rb'))
fhvhv_folderPath = pickle.load(open(v_direc + "fhvhv_folderPath",'rb'))

green_integratedPaths = pickle.load(open(v_direc + "green_integratedPaths",'rb'))
green_folderPath = pickle.load(open(v_direc + "green_folderPath",'rb'))

yellow_integratedPaths = pickle.load(open(v_direc + "yellow_integratedPaths",'rb'))
yellow_folderPath = pickle.load(open(v_direc + "yellow_folderPath",'rb'))

## General auxiliary functions
Auxiliary code to help in the data cleaning process goes here

In [8]:
def create_df(address):
# Takes csv from address given and creats a dataframe
# Returns given csv as a dataframe
# Schema of this csv will be in string format!!!!!
    # Reading a csv from address while taking headers as a schema
    # To infer schema type (i.e. int, float), add option inferSchema = True
        
    # When running in YARN mode, spark expects its input to come from HDFS by default
    # Copying files to local Hadoop would prevent a 'Path does not exist' error on the cluster
    # but this is not allowed
    # We can therefore only run the below in local mode, but there is a workaround through pandas
    
    # When opening these through pandas, we have to add the "error_bad_lines = False" argument
    # This is because PySpark seemed to drop rows with an extra column, but Pandas did not
    
    #Note - opening as Pandas appears to add a recordID in an unnamed first column
    
    # Code which does not work on cluster in HDFS mode:
    # Possibly solved by substituting HDFS file location above
    addressDF = spark.read.format('csv').option("header", True).load(address)
    
    print("opened", address)

    # Because we had to save as Pandas in 2.2, there is a new column _c0 in some dataframes which will interfere with operations
    # We need to detect, and drop, this column
    
    if '_c0' in addressDF.columns:
        addressDF = addressDF.drop(col('_c0'))
    
    
    # pandas_DF = pd.read_csv(address, warn_bad_lines = True, error_bad_lines = False)
    # addressDF = sql_sc.createDataFrame(pandas_DF.astype(str))
    
    return addressDF

In [9]:
def save_df(saveDF, saveName, savePath):
# Function takes a dataframe, filename, and filepath and saves the dataframe
# csv will be saved inside a folder with the given saveName, along with a text file _SUCCESS
# Want to use this as it is quicker than converting to PandasDF

# WARNING: WILL OVERWRITE PREVIOUS VERSIONS
    
    # I have not been given write permissions on the cluster and cannot directly save this through HDFS
    # Another workaround is to just move via pandas again
    # If working locally, the first line of code below can be uncommented and used instead
    
# NOTE: coalesce(1) can be used due to small data size - otherwise, must be written as multiple files due to memory restrictions 
    
    # Code which does not work on HDFS:
    saveDF.coalesce(1).write.options(header = True).mode("overwrite").csv(os.path.join(savePath, saveName))
    #saveDF.toPandas().to_csv(os.path.join(savePath, saveName))

In [10]:
def create_folder(newFolderName, newFolderPath):
# Function takes a folder name and a path and creates a new folder/directory
# Function returns folder path
    
    fullFolderPath = os.path.join(newFolderPath, newFolderName)
    
    # Folder might already have been created
    try:
        os.mkdir(fullFolderPath)
        print("new folder created at "+ fullFolderPath)
    except FileExistsError:
        print("folder already exists at " + fullFolderPath)
    
    return fullFolderPath

In [11]:
def create_df_fromSchema(schema):
# Takes a schema and creates an empty dataframe with that schema in string format
# Returns dataframe with schema and no values
#     fields = []
    
#     for i in schema:
#         fields.append(StructField(i, StringType(), True))
    
#     dfSchema = StructType(fields)
#     emptyDF = spark.createDataFrame([],dfSchema)
    emptyDF = spark.createDataFrame([],schema)
    
    return emptyDF

In [12]:
def drop_dupes(dupeData):
# function accepts a spark dataframe with duplicates
# function returns that dataframe with duplicates removed as [0]
# function returns the number of duplicates as [1]
# function returns the final length of the dataframe as [2]
    # we want to drop duplicates and track how many we catch - the native PySpark dataframe method drop_duplicates works()
    # it will drop based on all columns by default
    
    # NOTE: Pandas has added a _c0 to our dataframes upon saving - this disables functionality for drop_duplicates()
    # We need to drop _c0 before calling this function
    
    initialLen = dupeData.count()
    dupeData = dupeData.dropDuplicates()
    finalLen = dupeData.count()
    dupeCount = initialLen - finalLen
    print(str(dupeCount) + ' duplicate entries removed')
    
    return dupeData, dupeCount, finalLen

In [13]:
def cleanup(folderpath):
# Function which takes the folderpath for integrated data
# Function moves all CSV's to one folder and renames them properly
# Function returns the filepath of the new csvs
# DO NOT CLEAN MORE THAN ONCE, or your pickled variables will not work

    newPath = create_folder("all CSV", folderpath)
    testFilePath = []

    for (root,direc,files) in os.walk(folderpath):
        for file in files:
            if file.endswith('.csv'):
                if os.path.basename(root) != os.path.basename(newPath):
                    os.rename(os.path.join(root,file),os.path.join(newPath,os.path.basename(root)))
                    testFilePath.append(os.path.join(newPath,os.path.basename(root)))

    testFilePath.sort()
    return testFilePath

## Auxiliary functions for cleanup
Auxiliary code to help in the data cleaning process goes here
Please read the comments for a detailed description of validation criteria. We are using UDFs here, as the validation criteria are fairly complex. As mentioned though, their performance hit is minimal compared to the .count() calls.

In [14]:
def datetime_validation(pudt,dodt):
# Function to validate datetime values
# Function returns 0 if datetimes together are valid, 1 if they are invalid, 2 if they do not exist
# This function also checks that the timedelta is >=0
# We appear to need both for all queries with distance, so validation flags will be set together
    
    if pudt == "" or pudt == None or pudt =='nan' or dodt == "" or dodt == None or dodt =='nan':
        return 2
    
    # Convert to datetime to see if input follows the correct format
    try:
        pudt_DT = dt.strptime(pudt, '%Y-%m-%d %H:%M:%S')
        dodt_DT = dt.strptime(dodt, '%Y-%m-%d %H:%M:%S')
    except:
        return 1
    
    timeDelta = (dodt_DT - pudt_DT).total_seconds()
    # if the time difference is less than or equal to 0, we can return a 1
    if timeDelta <= 0:
        return 1
    
    # if timeDelta is above 0 and the formats are valid, we return a 0
    return 0

# Define function as a UDF for pyspark usage
udfdatetime_validation = udf(datetime_validation, IntegerType())

print("Date testing function stored as UDF")

def add_datetime_validation(dfDTval):
# Adds the datetime validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfDTval = dfDTval.withColumn('pickup_datetime_val', udfdatetime_validation('pickup_datetime',\
                                                                                'dropoff_datetime'))
    dfDTval = dfDTval.withColumn('dropoff_datetime_val', col('pickup_datetime_val'))

    return dfDTval

Date testing function stored as UDF


In [15]:
def location_validation(locID):
# Function to validate locationID values
# Function returns 0 if locationID is valid, 1 if it is invalid, 2 if it is None
# Function essentially checks that it is an integer and has a valid location code (above 0 and below 263)

    if locID == "" or locID == None or locID == 'nan':
        return 2

    # want to catch non-integer errors
    try:
        locID = int(locID)
    
        # Checking location code
        if locID <= 263 and locID >= 1:
            return 0
    except:
        return 1
    
    return 1

# Define function as a UDF for pyspark usage
udflocation_validation = udf(location_validation, IntegerType())

print("LocationID testing function stored as UDF")

def add_location_validation(dfLCval):
# Adds the location validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfLCval = dfLCval.withColumn('pulocationid_val', udflocation_validation('pulocationid'))
    dfLCval = dfLCval.withColumn('dolocationid_val', udflocation_validation('dolocationid'))

    return dfLCval

LocationID testing function stored as UDF


In [16]:
def srflag_validation(srflag):
# Function to validate sr_flag values
# Function returns 0 if sr_flag is valid, 1 if it is invalid, 2 if it is None
# SR Flag has to be either 1 or Null, which makes this simple

    # Checking sr_flag value - int(srflag) must be at the end or errors will appear
    if srflag == None or srflag == "" or srflag == 'nan':
        return 0
    if int(srflag) == 1:
        return 0
    return 1

# Define function as a UDF for pyspark usage
udfsr_validation = udf(srflag_validation, IntegerType())

print("sr_flag testing function stored as UDF")

def add_sr_validation(dfSRval):
# Adds the SR Flag validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfSRval = dfSRval.withColumn('sr_flag_val', udfsr_validation('sr_flag'))

    return dfSRval

sr_flag testing function stored as UDF


In [17]:
def basenum_validation(basenum):
# Function to validate dispatching_base_num values
# Function returns 0 if dispatching_base_num is valid, 1 if it is invalid, 2 if it is None
# Basenumbers seem to require format B00000 - the function checks for the length and the initial B
    
    if basenum == None or basenum == "" or basenum == 'nan':
        return 2
    
    # Checking overall length of base_num
    if len(basenum) == 6 and (basenum[0] == "B" or basenum[0] =="b"):
        return 0
    
    return 1
    
# Define function as a UDF for pyspark usage
udfbnum_validation = udf(basenum_validation, IntegerType())

print("dispatching_base_num testing function stored as UDF")

def add_basenum_validation(dfBNval):
# Adds the base number validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfBNval = dfBNval.withColumn('dispatching_base_num_val', udfbnum_validation('dispatching_base_num'))

    return dfBNval

dispatching_base_num testing function stored as UDF


In [18]:
def hvfhs_license_validation(licensenum):
# Function to validate hvfhs license number values
# Function returns 0 if the hvfhs number is valid, 1 if it is invalid, 2 if it is None
# Similarly to the basenumber, we can test for the four specific given values
    
    if licensenum == None or licensenum == "" or licensenum == 'nan':
        return 2
    
    if len(licensenum) == 6 and (licensenum == "HV0002" or licensenum == "HV0003"\
                                 or licensenum == "HV0004" or licensenum == "HV0005"):
        return 0
    
    return 1
    
# Define function as a UDF for pyspark usage
udflicensenum_validation = udf(hvfhs_license_validation, IntegerType())

print("hvfhs license number testing function stored as UDF")

def add_license_validation(dfLNval):
# Adds the hvfhs license number validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfLNval = dfLNval.withColumn('hvfhs_license_num_val', udflicensenum_validation('hvfhs_license_num'))

    return dfLNval

hvfhs license number testing function stored as UDF


In [19]:
def FHV_FHVHV_valid_row(pudt_val,dodt_val,puloc_val,doloc_val):
# takes input columns
# returns 1 if any of the columns contain 1
# returns 2 if any of the columns contain 2
# FHV and FHVHV have no amount columns - for the queries we are only interested in datetime and location

    if pudt_val == 1 or dodt_val == 1 or puloc_val == 1 or doloc_val == 1:
        return 1
    if pudt_val == 2 or dodt_val == 2 or puloc_val == 2 or doloc_val == 2:
        return 2

    return 0

# Define function as a UDF for pyspark usage
udfFHV_FHVHV_valid_row = udf(FHV_FHVHV_valid_row, IntegerType())

print("FHV & FHVHV row testing function stored as UDF")

def add_FHV_HV_row_validation(dfFHV_HVval):
# Adds the row validation as a new dataframe column, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications

    dfFHV_HVval = dfFHV_HVval.withColumn('row_val', udfFHV_FHVHV_valid_row('pickup_datetime_val',\
                                                                     'dropoff_datetime_val',\
                                                                     'pulocationid_val',\
                                                                     'dolocationid_val'))

    return dfFHV_HVval

FHV & FHVHV row testing function stored as UDF


# Cleaning datasets

### Analysis of the missing and dirty values

The obvious first step is identifying what information we actually need in our cleaned datasets. If a record is dirty in a dimension unnecessary for our analysis, there is no reason to drop the record.  

Our approach is simple: we want to minimze the loss of information but be able to compute each query.
Hence, at the begining of the cleaning of each sub-type of datasets (FHV,FHVHV,Green,Yellow), we state what
disqualifies a record from being part of the last type of files (what is referenced as "clean files" in the assignment pdf).

Here, our goal is to have usable datasets for our queries. Hence, if the data, after cleaning is such that no query can be computed, the record is discarded. Additionally, to maintain consistency, we will keep only data on which all queries can be run - we would not want to change the total number of trips across queries.

Info requried for the queries:

1. The number of trips: 
    - can be computed on the raw integrated files.
    - No impact on this part.
<br>
2. The number of trips in Manhatan and Brooklyn:
    - It is computable on each dataset (FHV, FHVHV, Green and Yellow).
    - Drop off and pick up location IDs required.
<br>
3. Monthly total receipt (exclude tips) and 
    - Computable only on the Green and Yellow dataset.
    - Requires All the monetary variables (or total and tip and possibly extra).
<br>
4. Average trip receipt (also exclude tips):
    - Computable only on the Green and Yellow dataset.
    - Requires All the monetary variables (or total and tip and possibly extra).
<br>
5. Average cost per in-progress minute:
    - Computable only on the Green and Yellow dataset.
    - Requires the tolls, surchages, fares, taxes and time (i.e. the difference between pick up and drop off datetime).
<br>
6. Average tip per trip:
    - Computable only on the Green and Yellow dataset.
    - Requires tip data.
<br>
7. Median monthly average speed per borough 
    - Computable only on the Green and Yellow dataset.
    - Requires the pudatetime and dodatetime, the distance ,the drop off and pick up locaiton IDs and the location IDs of each borough.
<br>
8. How long does it take to go to the airport:
    - Computable on each dataset(FHV,FHVHV,Green and Yellow).
    - Requires the drop off and pick up dates times as well as pu/do location IDs.
<br>

This gives us the following list of critical variables:

**FHV/FHVHV variables**
- pickup_datetime
- dropoff_datetime
- pulocationid
- dolocationid

**Green/Yellow variables**
- l/tpep_pickup_datetime
- l/tpep_dropoff_datetime
- trip_distance
- pulocationid 
- dolocationid  
- tip_amount
- total_amount
- payment_type

note: these following two do not appear in the data dictionary, but are unnecessary regardless.
- congestion_surcharge
- ehail fee


For the cost related queries, it is specified that Cost should exclude tips, but include fares, surcharges, taxes and tolls. The total_amount less the tip amount, as specified by the dictionary, will satisfy this condition, and so we will prioritise taking that variable in a complete form, and potentially repairing it with the others if it is missing. This has the added benefit that the total amount is already the most likely to be accurate, as it would need to be correct for customer billing purposes. For payment_type, we only need to filter out cash tips for one of our queries. There is, in this variable, also a line item for voided_trips and one for trips with no charge - we will remove all of these records before cleaning, as free trips should not be counted as a regular taxi journey. 

Accordingly, in the analysis below, we will detect all dirty variable entries according to validity constraints (as specified in the assignment), however dirty variables which are unnecessary will not cause a record to be dirty, as that record will of course still be usable.

We design testing functions that will flag any default for the relevant values. The data is supposed to respect specific formats according to https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
If a format is not respected for query-relevant data, we try to provide a solution by repairing the default.
If there is no solution to repair the record, we take steps depending on the overall picture.  

The procedure we generally try to follow is to work through the validation criteria, but in reverse. We count the number of each type of error found, and where positive we apply corrective action. Due to the need for counts, we have artificially slowed our code significantly. Otherwise, we generally will use PySpark UDFs for validation criteria, but PySpark native functions for identification and repair.

For each subset, we open each month's file and extract the broken data for repairs

## 1. Cleaning the FHV dataset

### Analysis of valid values and validity constraints

dispatching_base_num: TLC base license number - string, normally needs 6 characters, first character must be B or b (from observing data, not from data dictionary)  
pickup_datetime: 19 character string (needs 19 characters), should follow format yyyy-mm-dd hh:mm:ss  
dropoff_datetime: 19 character string (needs 19 characters), should follow format yyyy-mm-dd hh:mm:ss  
pulocationid: int 1 <= x <= 263  
dolocationid: int 1 <= x <= 263  
sr_flag: int 1 or null  

### Identifying and splitting off dirty records

Here, as in other taxi types, after first removing duplicates, we run a number of PySpark UDFs on each file to classify the clean and dirty variables. We will then split each file into a clean set and a dirty set, where the dirty set is placed in another dataframe for further analysis.  

The PySpark UDFs are unfortunate, performance wise, but due to the complexity of each validation requirement, they are more practical than native spark functions.

In the following, we will class a record (not a value) as being dirty if it is unusable for our purposes in 2.4 - the remaining clean records may have dirty values in unused variables, but are unnecessary for our final database.

If a key variable itself is dirty, we will attempt to use the remaining variables to repair it, but otherwise we will only track errors caught by our constraints.

For the queries:  
1. The monthly total number of trips
2. Monthly total number of trips in Manhattan and Brooklyn
3. The monthly total receipts - exclude tips, but include fares, surcharges, taxes and tolls
4. Average trip receipt
5. Average cost per in-progress minute
6. Average tip per trip
7. Median monthly average trip speed per borough
8. How long does it take to go to the airport?

We are unable to calculate 3, 4, 5, 6, and 7 by nature - the FHV data has no fare records. However, we can obviously calculate query 1, with pickup and dropoff intact we can calculate query 2, and with both pickup and dropoff times and location values we can calculate query 8.  

We could therefore consider records containing both pickup and dropoff locations or both locations and both times to still be useful. Records containing only one of them are not useful and as we have no further data, these records cannot be repaired. If row_val == 0, all four of these variables are included (for query 8) and we do not have to worry. We therefore can just also filter for records containing a pair of location values (query 2).

In [20]:
clean_FHV_directory = create_folder("cleaned FHV", save_directory)
with open(v_direc + "clean_FHV_directory",'wb') as cleanedfhvdirec:
    pickle.dump(clean_FHV_directory,cleanedfhvdirec)
    
dirty_FHV_directory = create_folder("dirty FHV", save_directory)
with open(v_direc + "dirty_FHV_directory",'wb') as dirtyfhvdirec:
    pickle.dump(dirty_FHV_directory,dirtyfhvdirec)

folder already exists at /home/epb123/output/cleaned FHV
folder already exists at /home/epb123/output/dirty FHV


In [21]:
def FHV_cleanup():
# takes no arguments - splits FHV into clean and dirty data
# returns a pyspark dataframe with dirty data and the clean filepaths

    dropped_duplicates_count = 0
    clean_count = 0
    row_count = 0
        
    for i in fhv_integratedPaths:
        dfi = create_df(i)
        dfi_row_count = dfi.count()
        row_count = row_count + dfi_row_count

        # Adding validation columns
        dfi = add_basenum_validation(dfi)
        dfi = add_datetime_validation(dfi)
        dfi = add_location_validation(dfi)
        dfi = add_sr_validation(dfi)
        dfi = add_FHV_HV_row_validation(dfi)

        # Store records which are not dirty (this does not consider variables useless for 2.4 completion)
        # We also want to keep records which we can use - namely those with a pair of locationID values
        dfiClean = dfi.filter((dfi.row_val == 0))
        
        dfiClean = dfiClean.drop(*['dispatching_base_num_val','pickup_datetime_val','dropoff_datetime_val','pulocationid_val', \
                              'dolocationid_val','sr_flag_val','row_val'])

        # We can drop duplicates in the clean dataset here - they will all have pickup and dropoff times or locations
        # The times are stored to the millisecond and PySpark's drop_duplicates compares all columns
        # If two records have pickup and dropoff times identical to the millisecond, we can safely drop them
        # If two records have both location ids identical and identical dispatch numbers, we can also safely drop them
        duplicates_result = drop_dupes(dfiClean)
        dfiClean = duplicates_result[0]

        dropped_duplicates_count = dropped_duplicates_count + duplicates_result[1]

        dfiClean_row_count = duplicates_result[2]
        clean_count = clean_count + dfiClean_row_count

        # Save the cleaned dataframes - pyspark will not save an empty dataframe correctly - it will have no schema!
        if dfiClean_row_count > 0:
            save_df(dfiClean,ntpath.basename(i) + 'clean', clean_FHV_directory)
        else:
            print("no remaining records - CSV not saved")

        dirtyInI = dfi.filter((dfi.row_val == 1) | (dfi.row_val == 2))\
            .withColumn("filename", lit(ntpath.basename(i)))
        
        # We then add the dirty records to a spark dataframe, with an additional column specifying file of origin
        if i == fhv_integratedPaths[0]:
            dfiDirty = dirtyInI

        else:
            dfiDirty = dfiDirty.unionByName(dirtyInI)
        
        save_df(dirtyInI,ntpath.basename(i) + 'dirty', dirty_FHV_directory)

        print(str(dfiClean_row_count) + ' clean records preserved out of ' + str(dfi_row_count) + \
              ' original records - ' + str(dfiClean_row_count/dfi_row_count*100)[0:4] + "%")
        print(str(dirtyInI.filter(dirtyInI.row_val == 1).count()) + " invalid records have been found")
        print(str(dirtyInI.filter(dirtyInI.row_val == 2).count()) + " records with missing data have been found")
        
        # We don't need them, but we can check for dirty sr_flag and basenumber values
        dfi = dfi.select([dfi.sr_flag_val, dfi.dispatching_base_num_val])
        
        print("For the variables useless for our 2.4 analysis, we have (among values not missing):")
        dfi.filter((dfi.sr_flag_val == 1)).\
                   agg(sql.sum("sr_flag_val")).show()
        dfi.filter((dfi.dispatching_base_num_val == 1)).\
                   agg(sql.sum("dispatching_base_num_val")).show()
        
                   
    print("Out of " + str(row_count) + " original records, " + str(clean_count) + " records were clean.")
    print(str(dropped_duplicates_count) + " records were dropped as duplicates.")
    print(str(row_count - clean_count - dropped_duplicates_count) \
          + " dirty records remain for resolution.")
    
    return dfiDirty

In [22]:
dirtyFHV = FHV_cleanup()

opened /home/epb123/output/integrated FHV/fhv_tripdata_2015-01.csv
0 duplicate entries removed
no remaining records - CSV not saved
0 clean records preserved out of 5462 original records - 0.0%
182 invalid records have been found
5280 records with missing data have been found
For the variables useless for our 2.4 analysis, we have (among values not missing):
+----------------+
|sum(sr_flag_val)|
+----------------+
|            null|
+----------------+

+-----------------------------+
|sum(dispatching_base_num_val)|
+-----------------------------+
|                           10|
+-----------------------------+

opened /home/epb123/output/integrated FHV/fhv_tripdata_2015-02.csv
0 duplicate entries removed
no remaining records - CSV not saved
0 clean records preserved out of 6220 original records - 0.0%
190 invalid records have been found
6030 records with missing data have been found
For the variables useless for our 2.4 analysis, we have (among values not missing):
+----------------+
|s

We end up keeping 660,590 (with 1 duplicate) records out of the original 1,389,608. We do end up dropping many months worth of data (everything up until 2017-06!) as a part of this, but given that the queries will rely on dodatetime or dolocationid, we do not have much of a choice. Imputing may be possible, but the accuracy would be so little as to be negligible. I would prefer to compute the first query of 2.4 on the full dataset, and then use this clean version for the rest - the pure number of trips should be clean (given the 2 duplicates picked up), but location based queries will require location data.  

This is unfortunate when comparing location and time based queries across months, but with no way of repairing due to the limited data in the FHV records, we must live with it. 

Additionally, it looks like there are a small number of invalid dispatching numbers in each file, and almost 6000 broken sr_flag values in one file. As we are far more interested in the other variables (in fact, we do not care about these at all), we will not consider invalid values in dispatching_base_num and sr_flag as something worth classifying a record as dirty.

In [23]:
# Let's look at our dirty record data
FHVdamaged_rows = dirtyFHV.filter(dirtyFHV.row_val == 1).count()
print("There were "+str(FHVdamaged_rows)+" damaged records")
      
FHVmissing_rows = dirtyFHV.filter(dirtyFHV.row_val == 2).count()
print("There were "+str(FHVmissing_rows)+" missing records")

There were 119436 damaged records
There were 609581 missing records


Without any other data - we cannot reasonably repair missing values. We proceed immediately to repairable records.

In [24]:
# we can check filter by row_val = 1 first --> by construction
# any row with a 1 anywhere will have taken row_val = 1
# these rows contain invalid, but not missing variable entries

def FHV_dirty_count(FHV1):
# Count corrupted records by variable type
    dirty_records = {}

    for i in range (6,len(FHV1.columns)-1):
        print('Checking ' + str(FHV1.columns[i]))
        tick = FHV1.filter(col(FHV1.columns[i]) == 1).count()
              
        if tick != 0:
            dirty_records[FHV1.columns[i]] = tick

    return dirty_records

# Let's start by seeing where the majority of errors lie (if there are any)
if FHVdamaged_rows != 0:
    print(FHV_dirty_count(dirtyFHV.filter(dirtyFHV.row_val == 1)))

Checking dispatching_base_num_val
Checking pickup_datetime_val
Checking dropoff_datetime_val
Checking pulocationid_val
Checking dolocationid_val
Checking sr_flag_val
Checking row_val
{'dispatching_base_num_val': 199, 'pickup_datetime_val': 11, 'dropoff_datetime_val': 11, 'pulocationid_val': 42154, 'dolocationid_val': 104905, 'sr_flag_val': 12, 'row_val': 119436}


In [25]:
# Store the dirty dataframe separately to avoid unfortunate errors
# We can update our cleaned values as we go

recleanedFHV = dirtyFHV

We can start with analysing the **dolocationid** and **pulocationid**

In [26]:
# Let's count our missing records first
print(str(dirtyFHV.filter(dirtyFHV.dolocationid_val == 2).count())+" missing dropoff values")
print(str(dirtyFHV.filter(dirtyFHV.pulocationid_val == 2).count())+" missing pickup values")

606462 missing dropoff values
282286 missing pickup values


Let's then start with **pulocationid**

In [27]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...

dirtyFHVanalysis = dirtyFHV.filter(col('pulocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyFHVanalysis.select("pulocationid",sql.col("pulocationid").cast("int").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


These would result from a "failed to match" failure coming out of the 2.2 locationid calculations. Basically, if the coordinates were corrupt - i.e. something was stored at 5 deg. longitude and 36 deg latitude, the shapefile would not identify the location as being in New York. We have none of these errors, so we should move on.  

We will have run the locationid testing code in 2.2, and the resulting errors must come from values 264, 265, and above.

In [28]:
print("We have "+str(recleanedFHV.filter(col('pulocationid') > 263).count())+" values above our threshold")
print("We have "+str(recleanedFHV.filter(col('pulocationid') == 0).count())+" zero values")
print("We have "+str(recleanedFHV.filter(col('pulocationid') < 0).count())+" values below our threshold")

We have 42036 values above our threshold
We have 118 zero values
We have 0 values below our threshold


We cannot repair these - all results coming from our location conversion would have fallen within our boundaries. All remaining results must be from later schemas without conversion.

In [29]:
recleanedFHV = recleanedFHV.withColumn('pulocationid', sql.when(col('pulocationid') > 263,\
                                            None).otherwise(col('pulocationid')))
recleanedFHV = recleanedFHV.withColumn('pulocationid', sql.when(col('pulocationid') == 0,\
                                            None).otherwise(col('pulocationid')))

In [30]:
print(str(recleanedFHV.filter(col('pulocationid') > 263).count())+" excess values remain after cleaning")
print(str(recleanedFHV.filter(col('pulocationid') == 0).count())+" zero values remain after cleaning")

0 excess values remain after cleaning
0 zero values remain after cleaning


Moving to **dolocationid**

In [31]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...

dirtyFHVanalysis = dirtyFHV.filter(col('dolocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyFHVanalysis.filter(col("dolocationid").cast("int").isNull()).count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [32]:
print("We have "+str(recleanedFHV.filter(col('dolocationid') > 263).count())+" values above our threshold")
print("We have "+str(recleanedFHV.filter(col('dolocationid') == 0).count())+" zero values")
print("We have "+str(recleanedFHV.filter(col('dolocationid') < 0).count())+" values below our threshold")

We have 103539 values above our threshold
We have 1366 zero values
We have 0 values below our threshold


In [33]:
recleanedFHV = recleanedFHV.withColumn('dolocationid', sql.when(col('dolocationid') > 263,\
                                            None).otherwise(col('dolocationid')))
recleanedFHV = recleanedFHV.withColumn('dolocationid', sql.when(col('dolocationid') == 0,\
                                            None).otherwise(col('dolocationid')))

In [34]:
print(str(recleanedFHV.filter(col('dolocationid') > 263).count())+" excess values remain after cleaning")
print(str(recleanedFHV.filter(col('dolocationid') == 0).count())+" zero values remain after cleaning")

0 excess values remain after cleaning
0 zero values remain after cleaning


We cannot do anything about the missing values with the data at hand, so let's move on.

Let's not forget to examine the incorrect **pickup/dropoff_datetime** columns

In [35]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyFHV.filter(dirtyFHV.dropoff_datetime_val == 1).count())+" dirty dropoff values and...")
print(str(dirtyFHV.filter(dirtyFHV.dropoff_datetime_val == 2).count())+" missing dropoff values")
print("We have "+str(dirtyFHV.filter(dirtyFHV.pickup_datetime_val == 1).count())+" dirty pickup values and...")
print(str(dirtyFHV.filter(dirtyFHV.pickup_datetime_val == 2).count())+" missing pickup values")

We have 11 dirty dropoff values and...
538883 missing dropoff values
We have 11 dirty pickup values and...
538883 missing pickup values


Here, the dirty values must come from negative trip distances or invalid formats. Let us test for both.

In [36]:
# We will here make the UDF such that it is applicable to FHV and FHVHV taxi types:
# If we have moved past our exclusion criteria, we want to see if we should swap the columns to repair the records

def datetime_switch_flag(pudt,dodt):
# We have just set invalid timestamps to Null, so
    if pudt == "" or pudt == None or pudt =='nan' or dodt == "" or dodt == None or dodt =='nan':
    # If Null, we don't want to switch them anyway
        return 0
    
    try:
        pudt_DT = dt.strptime(pudt, '%Y-%m-%d %H:%M:%S')
        dodt_DT = dt.strptime(dodt, '%Y-%m-%d %H:%M:%S')
    except:
        return 2
    # if the datetime records are in an invalid format, we can set null with a 2 return.
    
    timeDelta = (dodt_DT - pudt_DT).total_seconds()
    if timeDelta < 0:
        return 1
    # if the datetime records are 0, to the millisecond, we want to remove them as well!
    elif timeDelta == 0:
        return 2
    else:
        return 0

# Define function as a UDF for pyspark usage
udfdatetime_switch = udf(datetime_switch_flag, IntegerType())

print("FHV/FHVHV date testing function stored as UDF")

FHV/FHVHV date testing function stored as UDF


In [37]:
# Make the switch Flag dummy
recleanedFHV = recleanedFHV.withColumn('tsFlag',udfdatetime_switch('pickup_datetime','dropoff_datetime'))

print("We will now drop "+str(recleanedFHV.filter(col('tsFlag') == 2).count())+" invalid values")

recleanedFHV = recleanedFHV.withColumn('dropoff_datetime',sql.when(col('tsFlag') == 2, None).\
                                           otherwise(col('dropoff_datetime')))
recleanedFHV = recleanedFHV.withColumn('pickup_datetime',sql.when(col('tsFlag') == 2, None).\
                                           otherwise(col('pickup_datetime')))

print("We will now repair "+str(recleanedFHV.filter(col('tsFlag') == 1).count())+" backwards values")

# Put dropoff values into a temporary column
recleanedFHV = recleanedFHV.withColumn('dropoff_temp',col('dropoff_datetime'))

# Move pickup values into dropoff when switch flag is 1
recleanedFHV = recleanedFHV.withColumn('dropoff_datetime',sql.when(col('tsFlag') == 1,col('pickup_datetime')).\
                                           otherwise(col('dropoff_datetime')))
# Move dropoff values from temporary column into pickup when switch flag is 1
recleanedFHV = recleanedFHV.withColumn('pickup_datetime',sql.when(col('tsFlag') == 1,col('dropoff_temp')).\
                                           otherwise(col('pickup_datetime')))

# Drop temporary columns
recleanedFHV = recleanedFHV.drop(*['tsFlag','dropoff_temp'])

We will now drop 3 invalid values
We will now repair 8 backwards values


In [38]:
FHVpotential = dirtyFHV.filter((col('row_val') == 2) & (col('pulocationid_val') == 0) & (col('dolocationid_val') == 0)).count()
print("There are "+str(FHVpotential)+" values which we could possibly use to compute query 2 for FHV records.")

There are 8 values which we could possibly use to compute query 2 for FHV records.


Aside from our cleaning, we have investigated re-adding entries with ID values, but no times, as these could be used in Query 2 of t4.

Given the insignificance which re-adding these 8 rows would bring compared to the overall fully clean dataset (~700k) records, and the minor inconsistencies in the monthly number of fares resulting from their re-addition, we choose to not re-integrate this data.

In [39]:
# Run if saving as PySpark
FHV_cleanCSV_folderPath = cleanup(clean_FHV_directory)
with open(v_direc + "FHV_cleanCSV_folderPath",'wb') as cleanedfhvCSVdirec:
    pickle.dump(FHV_cleanCSV_folderPath,cleanedfhvCSVdirec)

FHV_dirtyCSV_folderPath = cleanup(dirty_FHV_directory)
with open(v_direc + "FHV_dirtyCSV_folderPath",'wb') as dirtyfhvCSVdirec:
    pickle.dump(FHV_dirtyCSV_folderPath,dirtyfhvCSVdirec)

folder already exists at /home/epb123/output/cleaned FHV/all CSV
folder already exists at /home/epb123/output/dirty FHV/all CSV


In [40]:
# We've set irreparable values as Null, so...
print("We have managed to repair "+str(recleanedFHV.filter(\
                                                             (col('pickup_datetime').isNotNull()) &\
                                                             (col('dropoff_datetime').isNotNull()) &\
                                                             (col('dolocationid').isNotNull()) &\
                                                             (col('pulocationid').isNotNull())).count())+" complete records")

We have managed to repair 1 complete records


It appears as if the massive number of unrepairable location errors means we cannot include even those repairable datetime values. We will add our repaired record to the relevant file, but we must discard 729,017 others.

In [41]:
# Let us filter for the values we want
everdirtyFHV = recleanedFHV.filter(\
                                     (col('pickup_datetime').isNull()) |\
                                     (col('dropoff_datetime').isNull()) |\
                                     (col('dolocationid').isNull()) |\
                                     (col('pulocationid').isNull())\
                                     )

recleanedFHV = recleanedFHV.filter(\
                                     (col('pickup_datetime').isNotNull()) &\
                                     (col('dropoff_datetime').isNotNull()) &\
                                     (col('dolocationid').isNotNull()) &\
                                     (col('pulocationid').isNotNull())\
                                     )

# Before re-integration, we must remember that we have re-cast a lot of columns as floats and the rest of the data is StringType
everdirtyFHV = everdirtyFHV.select([col(i).cast("string") for i in everdirtyFHV.columns])
recleanedFHV = recleanedFHV.select([col(i).cast("string") for i in recleanedFHV.columns])

# We can then drop the validation columns
recleanedFHV = recleanedFHV.drop(*['dispatching_base_num_val','pickup_datetime_val','dropoff_datetime_val','pulocationid_val', \
                              'dolocationid_val','sr_flag_val','row_val'])

# leaving us with just the filename variable as something which differentiates the schema from our clean data

In [42]:
def FHV_combine(cleanedupFHVDF,alwaysdirtyFHVDF):
    
    reAdded = 0
    for i in FHV_cleanCSV_folderPath:
        dfi = create_df(i)

        # We can here filter by filename to select the records which we would like to re-add
        # We can also add this if to skip the files which don't have any repaired records
        tempFilterDF = cleanedupFHVDF.filter(col('filename') == i.replace(clean_FHV_directory,"").\
                                                          replace('/all CSV/',"").\
                                                          replace('csvclean','csv'))
        if tempFilterDF.count() > 0:
            pre_existing = dfi.count()
            print(str(pre_existing)+" pre-existing clean records")

            dfi = dfi.unionByName(tempFilterDF.drop('filename'))

            # We drop duplicates in case the negative values were for accounting/record purposes (although this is unlikely)
            dupeTest = drop_dupes(dfi)
            dfi = dupeTest[0]
            addedDFI = dfi.count()-pre_existing
            print(str(addedDFI)+" additional records added")
            reAdded = reAdded + addedDFI
        else:
            print("File skipped - no cleaned up records")
            
        # For our cleanup later, let's save even unmodified files
        save_df(dfi,ntpath.basename(i), clean_FHV_directory)
        
    print(str(reAdded)+" records added across all files")
    
    if reAdded == 0:
        print("WARNING: REPAIRED RECORDS COULD NOT BE RE-ADDED - ORIGINAL FILE WAS EMPTY")
    
    # We have the dirty records saved on a monthly basis already anyway
    # We will therefore save everdirtyFHV alone
    # We will then have saved files per month with caught dirty records and one master file with unrecoverable records
    # This will record unrecoverable values, but will not take hours to run
    # We do not coalesce() here as the file is large (and we will not be reading it again outside of spark)
    
    alwaysdirtyFHVDF.write.options(header = True).mode("overwrite").\
         csv(os.path.join(dirty_FHV_directory, "irreparable_FHV_records"))

In [43]:
FHV_combine(recleanedFHV, everdirtyFHV)

opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-06.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-07.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-08.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-09.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-10.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-11.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2017-12.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2018-01.csvclean
File skipped - no cleaned up records
opened /home/epb123/output/cleaned FHV/all CSV/fhv_tripdata_2018-02.csvclean
File skippe

In [44]:
# Clean the PySpark outputs from before
FHV_cleanCSV_folderPath = cleanup(clean_FHV_directory)
with open(v_direc + "FHV_cleanCSV_folderPath",'wb') as cleanedFHVCSVdirec:
    pickle.dump(FHV_cleanCSV_folderPath,cleanedFHVCSVdirec)

# FHV_dirtyCSV_folderPath = cleanup(dirty_FHV_directory)
# with open(v_direc + "FHV_dirtyCSV_folderPath",'wb') as dirtyFHVCSVdirec:
#     pickle.dump(FHV_dirtyCSV_folderPath,dirtyFHVCSVdirec)

folder already exists at /home/epb123/output/cleaned FHV/all CSV


In [45]:
#We are deleting unused dataframes in order to preserve memory - we will need it for Yellow.
del recleanedFHV
del dirtyFHV
del dirtyFHVanalysis
del everdirtyFHV

## 2. Cleaning the FHVHV dataset

The FHVHV dataset is very similar to the FHV dataset - many of the same methods can therefore be re-used

### Analysis of valid values and validity constraints

Variables are identical to FHV apart from hvfhs_license_num!

hvfhs_license_num: a string either HV0002, HV0003, HV0004, HV0005  
dispatching_base_num: TLC base license number - string  
pickup_datetime: 2020-06-10 11:16:00  
dropoff_datetime: 2020-06-10 11:16:00  
pulocationid: int 1 to 263  
dolocationid: int 1 to 263  
sr_flag: int 1 or null
  

### Identifying and splitting off dirty records

In the following, we will class a record (not a value) as being dirty if it is unusable for our purposes in 2.4.

If the record itself is dirty, we will attempt to use the remaining variables to repair it, but otherwise we will only track errors caught by our constraints.

In [46]:
clean_FHVHV_directory = create_folder("cleaned FHVHV", save_directory)
with open(v_direc + "clean_FHVHV_directory",'wb') as cleanedfhvHVdirec:
    pickle.dump(clean_FHVHV_directory,cleanedfhvHVdirec)
    
dirty_FHVHV_directory = create_folder("dirty FHVHV", save_directory)
with open(v_direc + "dirty_FHVHV_directory",'wb') as dirtyfhvHVdirec:
    pickle.dump(dirty_FHVHV_directory,dirtyfhvHVdirec)

folder already exists at /home/epb123/output/cleaned FHVHV
folder already exists at /home/epb123/output/dirty FHVHV


The function to split FHVHV can be almost identical to that for FHV - the only additional variable is the license number, which plays no role in our intended analysis.

In [47]:
def FHVHV_cleanup():
# takes no arguments - splits FHV into clean and dirty data
# returns a pyspark dataframe with dirty data

    dropped_duplicates_count = 0
    clean_count = 0
    row_count = 0
        
    for i in fhvhv_integratedPaths:
        dfi = create_df(i)
        dfi_row_count = dfi.count()
        row_count = row_count + dfi_row_count

        # Adding validation columns
        dfi = add_license_validation(dfi)
        dfi = add_basenum_validation(dfi)
        dfi = add_datetime_validation(dfi)
        dfi = add_location_validation(dfi)
        dfi = add_sr_validation(dfi)
        dfi = add_FHV_HV_row_validation(dfi)

        # Store records which are not dirty (this does not consider variables useless for 2.4 completion)
        # We also want to keep records which we can use - namely those with a pair of locationID values
        dfiClean = dfi.filter((dfi.row_val == 0))
        
        dfiClean = dfiClean.drop(*['hvfhs_license_num_val','dispatching_base_num_val','pickup_datetime_val','dropoff_datetime_val','pulocationid_val', \
                              'dolocationid_val','sr_flag_val','row_val'])

        # We can drop duplicates in the clean dataset here - they will all have pickup and dropoff times or locations
        # The times are stored to the millisecond and PySpark's drop_duplicates compares all columns
        # If two records have pickup and dropoff times identical to the millisecond, we can safely drop them
        # If two records have both location ids identical and identical dispatch numbers, we can also safely drop them
        duplicates_result = drop_dupes(dfiClean)
        dfiClean = duplicates_result[0]

        dropped_duplicates_count = dropped_duplicates_count + duplicates_result[1]

        dfiClean_row_count = duplicates_result[2]
        clean_count = clean_count + dfiClean_row_count

        # Save the cleaned dataframes - spark will save an empty dataframe, but it will have no schema!
        if dfiClean_row_count > 0:
            save_df(dfiClean,ntpath.basename(i) + 'clean', clean_FHVHV_directory)
        else:
            print("no remaining records - CSV not saved")

        dirtyInI = dfi.filter((dfi.row_val == 1) | (dfi.row_val == 2))\
            .withColumn("filename", lit(ntpath.basename(i)))
        
        # We then add the dirty records to a spark dataframe, with an additional column specifying file of origin
        if i == fhvhv_integratedPaths[0]:
            dfiDirty = dirtyInI

        else:
            dfiDirty = dfiDirty.unionByName(dirtyInI)

        save_df(dirtyInI,ntpath.basename(i) + 'dirty', dirty_FHVHV_directory)
        
        print(str(dfiClean_row_count) + ' clean records preserved out of ' + str(dfi_row_count) + \
              ' original records - ' + str(dfiClean_row_count/dfi_row_count*100)[0:4] + "%")
        print(str(dirtyInI.filter(dirtyInI.row_val == 1).count()) + " invalid records have been found")
        print(str(dirtyInI.filter(dirtyInI.row_val == 2).count()) + " records with missing data have been found")
        
        # We don't need them, but we can check for dirty sr_flag, license number and basenumber values
        dfi = dfi.select([dfi.sr_flag_val, dfi.dispatching_base_num_val, dfi.hvfhs_license_num_val])
        
        print("For the variables useless for our 2.4 analysis, we have (among values not missing):")
        dfi.filter((dfi.sr_flag_val == 1)).\
                   agg(sql.sum("sr_flag_val")).show()
        dfi.filter((dfi.dispatching_base_num_val == 1)).\
                   agg(sql.sum("dispatching_base_num_val")).show()
        dfi.filter((dfi.hvfhs_license_num_val == 1)).\
                   agg(sql.sum("hvfhs_license_num_val")).show()
                   
    print("Out of " + str(row_count) + " original records, " + str(clean_count) + " records were clean.")
    print(str(dropped_duplicates_count) + " records were dropped as duplicates.")
    print(str(row_count - clean_count - dropped_duplicates_count) \
          + " dirty records remain for resolution.")
    
    return dfiDirty

In [48]:
dirtyFHVHV = FHVHV_cleanup()

opened /home/epb123/output/integrated FHVHV/fhvhv_tripdata_2019-02.csv
0 duplicate entries removed
39089 clean records preserved out of 40290 original records - 97.0%
1201 invalid records have been found
0 records with missing data have been found
For the variables useless for our 2.4 analysis, we have (among values not missing):
+----------------+
|sum(sr_flag_val)|
+----------------+
|            null|
+----------------+

+-----------------------------+
|sum(dispatching_base_num_val)|
+-----------------------------+
|                         null|
+-----------------------------+

+--------------------------+
|sum(hvfhs_license_num_val)|
+--------------------------+
|                      null|
+--------------------------+

opened /home/epb123/output/integrated FHVHV/fhvhv_tripdata_2019-03.csv
0 duplicate entries removed
46287 clean records preserved out of 47703 original records - 97.0%
1416 invalid records have been found
0 records with missing data have been found
For the variables

In [49]:
# Store the dirty dataframe separately to avoid unfortunate errors
# We can update our cleaned values as we go

recleanedFHVHV = dirtyFHVHV

In [50]:
FHVHVdamaged_rows = dirtyFHVHV.filter(dirtyFHVHV.row_val == 1).count()
print(str(FHVHVdamaged_rows) + ' rows have damaged (not missing) data')

# we can check filter by row_val = 1 first --> by construction
# any row with a 1 anywhere will have taken row_val = 1
# these rows contain invalid, but not missing variable entries

def FHVHV_dirty_count(FHVHV1):
# Count corrupted records by variable type
    dirty_records = {}

    for i in range (7,len(FHVHV1.columns)-1):
        print('Checking ' + str(FHVHV1.columns[i]))
        tick = FHVHV1.filter(col(FHVHV1.columns[i]) == 1).count()
              
        if tick != 0:
            dirty_records[FHVHV1.columns[i]] = tick

    return dirty_records

# Let's start by seeing where the majority of errors lie (if there are any)
if FHVHVdamaged_rows != 0:
    print(FHVHV_dirty_count(dirtyFHVHV.filter(dirtyFHVHV.row_val == 1)))

9811 rows have damaged (not missing) data
Checking hvfhs_license_num_val
Checking dispatching_base_num_val
Checking pickup_datetime_val
Checking dropoff_datetime_val
Checking pulocationid_val
Checking dolocationid_val
Checking sr_flag_val
Checking row_val
{'pickup_datetime_val': 1, 'dropoff_datetime_val': 1, 'pulocationid_val': 20, 'dolocationid_val': 9796, 'row_val': 9811}


We have very little errors aside from the dolocationID. As we need dolocationid for queries 2 and 8, we hope to repair them.

We can start with analysing the **dolocationid** and **pulocationid**

In [51]:
# Let's count our missing records first
print(str(dirtyFHVHV.filter(dirtyFHVHV.dolocationid_val == 2).count())+" missing dropoff values")
print(str(dirtyFHVHV.filter(dirtyFHVHV.pulocationid_val == 2).count())+" missing pickup values")

0 missing dropoff values
0 missing pickup values


Let's then start with **pulocationid**

In [52]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...

dirtyFHVHVanalysis = dirtyFHVHV.filter(col('pulocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyFHVHVanalysis.select("pulocationid",sql.col("pulocationid").cast("int").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [53]:
print("We have "+str(recleanedFHVHV.filter(col('pulocationid') > 263).count())+" values above our threshold")
print("We have "+str(recleanedFHVHV.filter(col('pulocationid') == 0).count())+" zero values")
print("We have "+str(recleanedFHVHV.filter(col('pulocationid') < 0).count())+" values below our threshold")

We have 20 values above our threshold
We have 0 zero values
We have 0 values below our threshold


We cannot repair these - all results coming from our location conversion would have fallen within our boundaries. All remaining results must be from later schemas without conversion.

In [54]:
recleanedFHVHV = recleanedFHVHV.withColumn('pulocationid', sql.when(col('pulocationid') > 263,\
                                            None).otherwise(col('pulocationid')))

In [55]:
print(str(recleanedFHVHV.filter(col('pulocationid') > 263).count())+" excess values remain after cleaning")

0 excess values remain after cleaning


Moving to **dolocationid**

In [56]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...

dirtyFHVHVanalysis = dirtyFHVHV.filter(col('dolocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyFHVHVanalysis.filter(col("dolocationid").cast("int").isNull()).count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [57]:
print("We have "+str(recleanedFHVHV.filter(col('dolocationid') > 263).count())+" values above our threshold")
print("We have "+str(recleanedFHVHV.filter(col('dolocationid') == 0).count())+" zero values")
print("We have "+str(recleanedFHVHV.filter(col('dolocationid') < 0).count())+" values below our threshold")

We have 9796 values above our threshold
We have 0 zero values
We have 0 values below our threshold


In [58]:
recleanedFHVHV = recleanedFHVHV.withColumn('dolocationid', sql.when(col('dolocationid') > 263,\
                                            None).otherwise(col('dolocationid')))

In [59]:
print(str(recleanedFHVHV.filter(col('dolocationid') > 263).count())+" excess values remain after cleaning")

0 excess values remain after cleaning


We cannot do anything about the missing values with the data at hand, so let's move on.

Let's not forget to examine the incorrect **pickup/dropoff_datetime** columns

In [60]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyFHVHV.filter(dirtyFHVHV.dropoff_datetime_val == 1).count())+" dirty dropoff values and...")
print(str(dirtyFHVHV.filter(dirtyFHVHV.dropoff_datetime_val == 2).count())+" missing dropoff values")
print("We have "+str(dirtyFHVHV.filter(dirtyFHVHV.pickup_datetime_val == 1).count())+" dirty pickup values and...")
print(str(dirtyFHVHV.filter(dirtyFHVHV.pickup_datetime_val == 2).count())+" missing pickup values")

We have 1 dirty dropoff values and...
0 missing dropoff values
We have 1 dirty pickup values and...
0 missing pickup values


Here, the dirty values must come from negative trip distances or invalid formats. Let us test for both.

In [61]:
# Make the switch Flag dummy
recleanedFHVHV = recleanedFHVHV.withColumn('tsFlag',udfdatetime_switch('pickup_datetime','dropoff_datetime'))

print("We will now drop "+str(recleanedFHVHV.filter(col('tsFlag') == 2).count())+" invalid values")

recleanedFHVHV = recleanedFHVHV.withColumn('dropoff_datetime',sql.when(col('tsFlag') == 2, None).\
                                           otherwise(col('dropoff_datetime')))
recleanedFHVHV = recleanedFHVHV.withColumn('pickup_datetime',sql.when(col('tsFlag') == 2, None).\
                                           otherwise(col('pickup_datetime')))

print("We will now repair "+str(recleanedFHVHV.filter(col('tsFlag') == 1).count())+" backwards values")

# Put dropoff values into a temporary column
recleanedFHVHV = recleanedFHVHV.withColumn('dropoff_temp',col('dropoff_datetime'))

# Move pickup values into dropoff when switch flag is 1
recleanedFHVHV = recleanedFHVHV.withColumn('dropoff_datetime',sql.when(col('tsFlag') == 1,col('pickup_datetime')).\
                                           otherwise(col('dropoff_datetime')))
# Move dropoff values from temporary column into pickup when switch flag is 1
recleanedFHVHV = recleanedFHVHV.withColumn('pickup_datetime',sql.when(col('tsFlag') == 1,col('dropoff_temp')).\
                                           otherwise(col('pickup_datetime')))

# Drop temporary columns
recleanedFHVHV = recleanedFHVHV.drop(*['tsFlag','dropoff_temp'])

We will now drop 1 invalid values
We will now repair 0 backwards values


In [62]:
FHVHVpotential = dirtyFHVHV.filter((col('row_val') == 2) & (col('pulocationid_val') == 0) & (col('dolocationid_val') == 0)).count()
print("There are "+str(FHVHVpotential)+" values which we could possibly use to compute query 2 for FHVHV records.")

There are 0 values which we could possibly use to compute query 2 for FHVHV records.


In [63]:
# Run if saving as PySpark
FHVHV_cleanCSV_folderPath = cleanup(clean_FHVHV_directory)
with open(v_direc + "FHVHV_cleanCSV_folderPath",'wb') as cleanedFHVHVCSVdirec:
    pickle.dump(FHVHV_cleanCSV_folderPath,cleanedFHVHVCSVdirec)

FHVHV_dirtyCSV_folderPath = cleanup(dirty_FHVHV_directory)
with open(v_direc + "FHVHV_dirtyCSV_folderPath",'wb') as dirtyFHVHVCSVdirec:
    pickle.dump(FHVHV_dirtyCSV_folderPath,dirtyFHVHVCSVdirec)

folder already exists at /home/epb123/output/cleaned FHVHV/all CSV
folder already exists at /home/epb123/output/dirty FHVHV/all CSV


In [64]:
# We've set irreparable values as Null, so...
print("We have managed to repair "+str(recleanedFHVHV.filter(\
                                                             (col('pickup_datetime').isNotNull()) &\
                                                             (col('dropoff_datetime').isNotNull()) &\
                                                             (col('dolocationid').isNotNull()) &\
                                                             (col('pulocationid').isNotNull())).count())+" complete records")

We have managed to repair 0 complete records


It appears as if the massive number of unrepairable location errors means we cannot include even those repairable datetime values, and we do not want to add incomplete records to our clean dataset - we will save our dirty records and move on to Green, having lost all 9,811 dirty records.

In [65]:
# Let us save the dirty dataset   
recleanedFHVHV.write.options(header = True).mode("overwrite").csv(os.path.join(dirty_FHVHV_directory, "irreparable_FHVHV_records"))

In [66]:
del recleanedFHVHV
del dirtyFHVHV
del dirtyFHVHVanalysis

## Auxiliary functions for cleanup of Green and Yellow
Auxiliary code to help in the data cleaning process goes here

We need further functions to check the Green and Yellow datasets, based on their greater number of variables. We again rely on UDFs as again - the expense is worth the convenience.

In [67]:
def improvement_validation_post2015(imp):
# Function to validate improvement surcharge float values
# Function returns 0 if the figure is valid, 1 if it is invalid, 2 if it is None
# This just tests the validation is not a string and is equal to either 0 or 0.3, the values given by the data dictionary.
    
    if imp == None or imp == "" or imp == "nan":
        return 2
    
    try:
        imp = float(imp)
        if imp == 0 or imp == 0.3:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfimp_validation_post2015 = udf(improvement_validation_post2015, IntegerType())

print("Post-2015 improvement surcharge testing function stored as UDF")

def improvement_validation_pre2015(imp):
# Function to validate improvement surcharge float values
# Function returns 0 if the figure is valid, 1 if it is invalid
    
    if imp == None or imp == "" or imp == 'nan':
        return 0
    
    try:
        if float(imp) == 0:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfimp_validation_pre2015 = udf(improvement_validation_pre2015, IntegerType())

print("Pre-2015 improvement surcharge testing function stored as UDF")   

def add_improvement_surcharge_validation(dfIMPval, pathaddress):
# Adds the improvement surcharge validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    if int(pathaddress[-11:-7]) < 2015:
        dfIMPval = dfIMPval.withColumn('improvement_surcharge_val', udfimp_validation_pre2015('improvement_surcharge'))
    else:                                      
        dfIMPval = dfIMPval.withColumn('improvement_surcharge_val', udfimp_validation_post2015('improvement_surcharge'))
                                        
    return dfIMPval                               

Post-2015 improvement surcharge testing function stored as UDF
Pre-2015 improvement surcharge testing function stored as UDF


In [68]:
def tax_validation(tax):
# Function to validate mta tax float values
# Function returns 0 if the figure is valid, 1 if it is invalid, 2 if it is None
# Tax has to be 0 or 0.5
    
    if tax == None or tax == "" or tax == 'nan':
        return 2
    
    try:
        tax = float(tax)
        if tax == 0 or tax == 0.5:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udftax_validation = udf(tax_validation, IntegerType())

print("tax testing function stored as UDF")

def add_mta_tax_validation(dfTXval):
# Adds the tax validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfTXval = dfTXval.withColumn('mta_tax_val', udftax_validation('mta_tax'))

    return dfTXval

tax testing function stored as UDF


In [69]:
def extra_validation(extra):
# Function to validate extra float values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# Extra has to be the values specified below, in addition to other criteria
    
    if extra == None or extra == "" or extra == 'nan':
        return 2
    
    try:
        extra = float(extra)
    
        # Want to flag values above $1.50 - data dictionary says $1.50 should be the max
        if extra == 0 or extra == 0.5 or extra == 1 or extra == 1.5:
            return 0
    
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfextra_validation = udf(extra_validation, IntegerType())

print("extra testing function stored as UDF")

def add_extra_validation(dfEXval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfEXval = dfEXval.withColumn('extra_val', udfextra_validation('extra'))

    return dfEXval

extra testing function stored as UDF


In [70]:
def money_validation(cash):
# Function to validate money/transaction float values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# Want to flag values above 1000 - fares, tips, tolls, totals shouldn't be above 1000
# items should be able to = 0, in particular tolls and tip
# Due to their similarity, the same UDF can be used for multiple of the money-based variables.

    if cash == None or cash == "" or cash == 'nan':
        return 2
    
    try:
        cash = float(cash)

        if cash >= 0 and cash <= 1000:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfmoney_validation = udf(money_validation, IntegerType())

print("money testing function stored as UDF")

def add_fare_amount_validation(dfCASHval):
# Adds the specified validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfCASHval = dfCASHval.withColumn('fare_amount_val', udfmoney_validation('fare_amount'))

    return dfCASHval

def add_tip_amount_validation(dfCASHval):
# Adds the specified validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfCASHval = dfCASHval.withColumn('tip_amount_val', udfmoney_validation('tip_amount'))

    return dfCASHval

def add_tolls_amount_validation(dfCASHval):
# Adds the specified validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfCASHval = dfCASHval.withColumn('tolls_amount_val', udfmoney_validation('tolls_amount'))

    return dfCASHval

money testing function stored as UDF


In [71]:
def sflag_validation(sflag):
# Function to validate store_and_fwd_flag values
# Function returns 0 if the flag is valid, 1 if it is invalid, 2 if it is None
    
    if sflag == None or sflag == "" or sflag == "nan":
        return 2
    
    if sflag == "Y" or sflag == "N":
        return 0
    
    return 1
    
# Define function as a UDF for pyspark usage
udfsflag_validation = udf(sflag_validation, IntegerType())

print("store-and-forward-flag testing function stored as UDF")

def add_store_and_forward_flag_validation(dfFLAGval):
# Adds the store and forward flag validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfFLAGval = dfFLAGval.withColumn('store_and_fwd_flag_val', udfsflag_validation('store_and_fwd_flag'))

    return dfFLAGval

store-and-forward-flag testing function stored as UDF


In [72]:
def distance_validation(tdist):
# Function to validate trip distance values
# Function returns 0 if the trip distance is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# Want to flag values above 300 - NYC to Boston/DC is ~300 miles and some people might use taxis for this
# Anything above that is probably an error
# Values =0 are not trips - probably errors

    if tdist == None or tdist == "" or tdist == "nan":
        return 2
    
    try:
        tdist = float(tdist)

        if tdist > 0 and tdist <= 300:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfdistance_validation = udf(distance_validation, IntegerType())

print("trip distance testing function stored as UDF")

def add_trip_distance_validation(dfDISTval):
# Adds the trip distance validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfDISTval = dfDISTval.withColumn('trip_distance_val', udfdistance_validation('trip_distance'))

    return dfDISTval

trip distance testing function stored as UDF


In [73]:
def passenger_count_validation(pcount):
# Function to validate passenger count values
# Function returns 0 if the passenger count is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# Want to flag values above 10 - it's probably an error if >10 people are in the same taxi
    
    if pcount == None or pcount == "" or pcount == "nan":
        return 2
    
    # want to catch non-integer errors
    try:
        pcount = int(pcount)
        if pcount > 0 and pcount <= 10:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfpassengercount_validation = udf(passenger_count_validation, IntegerType())

print("passenger count testing function stored as UDF")

def add_passenger_count_validation(dfPASSval):
# Adds the passenger count validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfPASSval = dfPASSval.withColumn('passenger_count_val', udfpassengercount_validation('passenger_count'))

    return dfPASSval

passenger count testing function stored as UDF


In [74]:
def vendorid_triptype_validation(vidttype):
# Function to validate vendorid and trip type values
# Function returns 0 if the vendorid or trip type is valid, 1 if it is invalid, 2 if it is None
# We know from the dictionary that VendorID must be either 1 or 2
    
    if vidttype == None or vidttype == "" or vidttype == 'nan':
        return 2
    
    # want to catch non-integer errors
    try:
        vidttype = int(vidttype)
        if vidttype == 1 or vidttype == 2:
            return 0
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfvendoridTripType_validation = udf(vendorid_triptype_validation, IntegerType())

print("vendorid and trip type testing function stored as UDF")

def add_vendorid_validation(dfVIDval):
# Adds the vendorid validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfVIDval = dfVIDval.withColumn('vendorid_val', udfvendoridTripType_validation('vendorid'))

    return dfVIDval

def add_trip_type_validation(dfTTval):
# Adds the trip_type validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfTTval = dfTTval.withColumn('trip_type_val', udfvendoridTripType_validation('trip_type'))

    return dfTTval

vendorid and trip type testing function stored as UDF


In [75]:
def payment_validation(paymentrate,tip):
# Function to validate payment type and ratecodeid values
# Function returns 0 if the payment type is valid, 1 if it is invalid, 2 if it is None
# Explanations on the filtering mechanism follow below
    
    if paymentrate == None or paymentrate == "" or paymentrate == 'nan':
        return 2
    
    # Flag non-integers
    try:
        paymentrate = int(paymentrate)   
    except:
        return 1

    # Flag invalid values
    if paymentrate < 1 or paymentrate > 6:
        return 1
    
    # if Payment_type is 3 or 6, we will flag the value (No charge or voided trip)
    if paymentrate == 3 or paymentrate == 6:
        return 1
    
    # If the payment_type corresponds to a cash payment, but the tip is positive, we will trust the tip entry
    # We do this because the tip record is almost certainly more reliable than the cash/card button on the machine.
    try:
        tip = abs(float(tip))
        if paymentrate == 2 and tip != 0:
                return 1
    except:
        # If we can't check the tip due to an incorrect string, return 0
        return 0
    
    return 0
    
# Define function as a UDF for pyspark usage
udfpaymentType_validation = udf(payment_validation, IntegerType())

def add_payment_type_validation(dfPAYval):
# Adds the payment type validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfPAYval = dfPAYval.withColumn('payment_type_val', udfpaymentType_validation('payment_type','tip_amount'))

    return dfPAYval

def ratecode_validation(paymentrate):
# Function to validate payment type and ratecodeid values
# Function returns 0 if the payment type is valid, 1 if it is invalid, 2 if it is None
    
    if paymentrate == None or paymentrate == "" or paymentrate == 'nan':
        return 2
    
    # Flag non-integers
    try:
        paymentrate = int(paymentrate)
    
        if paymentrate >= 1 and paymentrate <= 6:
            return 0
    
    except:
        return 1
    
    return 1
    
# Define function as a UDF for pyspark usage
udfRatecodeID_validation = udf(ratecode_validation, IntegerType())

def add_ratecodeID_validation(dfRIDval):
# Adds the ratecode ID validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfRIDval = dfRIDval.withColumn('ratecodeid_val', udfRatecodeID_validation('ratecodeid'))

    return dfRIDval

print("payment type and ratecodeid testing function stored as UDF")

payment type and ratecodeid testing function stored as UDF


In [76]:
def ehail_validation(ehail):
# Function to validate ehail_fee values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
    
    if ehail == None or ehail == "" or ehail == 'nan':
        return 0
    
    return 1
    
# Define function as a UDF for pyspark usage
udfehail_validation = udf(ehail_validation, IntegerType())

print("ehail testing function stored as UDF")

def add_ehail_validation(dfEHval):
# Adds the ehail validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfEHval = dfEHval.withColumn('ehail_fee_val', udfehail_validation('ehail_fee'))

    return dfEHval

ehail testing function stored as UDF


In [77]:
def GREENcongestion_surcharge_validation(cs):
# Function to validate congestion_surcharge values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# This was not in the Data Dictionary, however the data appears to require 2.75 for green - we will explore this with the UDF

    if cs == None or cs == "" or cs == 'nan':
        return 2
    
    try:
        cs = float(cs)
    
        # Want to flag values not = 2.75 or = 0
        if cs == 2.75 or cs == 0:
            return 0
    
    except:
        return 1
    
    return 1

# Define function as a UDF for pyspark usage
udfGREENCS_validation = udf(GREENcongestion_surcharge_validation, IntegerType())

print("Green Congestion Surcharge testing function stored as UDF")

def add_GREEN_congestion_surcharge_validation(dfCSval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfCSval = dfCSval.withColumn('congestion_surcharge_val', udfGREENCS_validation('congestion_surcharge'))

    return dfCSval

def YELLOWcongestion_surcharge_validation(cs):
# Function to validate congestion_surcharge values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# This was not in the Data Dictionary, however the data appears to require 2.5 for Yellow - we will explore this with the UDF

    if cs == None or cs == "" or cs == 'nan':
        return 2
    
    try:
        cs = float(cs)
    
        # Want to flag values not = 2.5 or = 0
        if cs == 2.5 or cs == 0:
            return 0
    
    except:
        return 1
    
    return 1

# Define function as a UDF for pyspark usage
udfYELLOWCS_validation = udf(YELLOWcongestion_surcharge_validation, IntegerType())

print("Yellow Congestion Surcharge testing function stored as UDF")

def add_YELLOW_congestion_surcharge_validation(dfCSval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfCSval = dfCSval.withColumn('congestion_surcharge_val', udfYELLOWCS_validation('congestion_surcharge'))

    return dfCSval

Green Congestion Surcharge testing function stored as UDF
Yellow Congestion Surcharge testing function stored as UDF


In [78]:
def GREENreceipt_val(total,cong_s,extra,fare,improv,tax,tip,toll,ehail):
# Function to validate total_amount values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# Essentially - we want to flag how many total_amounts are too far away from the sum of their components
# We will of course be slightly arbitrary with this, but this is necessary
# We settle on classifying as worth investigation all records with a discrepancy greater than 5% of the total value
# We ignore here ehail as it is blank
# We are taking the absolute value in order to avoid negative errors, which we will correct for later ahead of the cash filter
    
    if total == None or total == "" or total == 'nan':
        return 2
    
    try:
        total = float(total)
    except:
        return 1
    
    # we use absolute values here because we do not want incorrectly stored negatives to impact our filter
    difference = abs(total)
    
    for i in [cong_s,extra,fare,improv,tax,tip,toll,ehail]:
        if i == None or i == "" or i == 'nan' or i == 'NaN':
            i = 0
        try:
            # we also do not want to let incorrectly stored strings get in our way
            i = abs(float(i))
            difference = difference - i
        except:
            pass
    
    # we needed a percentage figure here, and the 5% from a standard significance test seemed appropriate
    # this will allow us to allow some leeway in the values of the other variables while still flagging what would be
    # an incorrect calculation
    # A trip of 0 should theoretically be possible - enter and leave the taxi with no movement
    
    if abs(difference) <= abs(total)*0.05 and total >= 0 and total <= 1000:

        return 0
    
    return 1

# Define function as a UDF for pyspark usage
udfGREENreceipt_val = udf(GREENreceipt_val, IntegerType())

print("Green receipt validation testing function stored as UDF")

def add_GREEN_total_amount_validation(dfrval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfrval = dfrval.withColumn('total_amount_val', udfGREENreceipt_val('total_amount','congestion_surcharge','extra',\
    'fare_amount','improvement_surcharge','mta_tax','tip_amount','tolls_amount','ehail_fee'))

    return dfrval

Green receipt validation testing function stored as UDF


In [79]:
def YELLOWreceipt_val(total,cong_s,extra,fare,improv,tax,tip,toll):
# Function to validate total_amount values
# Function returns 0 if the figure is valid, 1 if it is invalid OR SUSPICIOUS, 2 if it is None
# Essentially - we want to flag how many total_amounts are too far away from the sum of their components
# We will of course be slightly arbitrary with this, but this is necessary
# We settle on classifying as worth investigation all records with a discrepancy greater than 5% of the total value
# We ignore here ehail as it is blank
# We are taking the absolute value in order to 

    if total == None or total == "" or total == 'nan':
        return 2
    
    try:
        total = float(total)
    except:
        return 1
    
    # we use absolute values here because we do not want incorrectly stored negatives to impact our filter
    difference = abs(total)
    
    for i in [cong_s,extra,fare,improv,tax,tip,toll]:
        if i == None or i == "" or i == 'nan' or i == 'NaN':
            i = 0
        try:
            # we also do not want to let incorrectly stored strings get in our way
            i = abs(float(i))
            difference = difference - i
        except:
            pass
    
    # we needed a percentage figure here, and the 5% from a standard significance test seemed appropriate
    # this will allow us to allow some leeway in the values of the other variables while still flagging what would be
    # an incorrect calculation
    
    if abs(difference) <= abs(total)*0.05 and total >= 0 and total <= 1000:

        return 0
    
    return 1

# Define function as a UDF for pyspark usage
udfYELLOWreceipt_val = udf(YELLOWreceipt_val, IntegerType())

print("YELLOW receipt validation testing function stored as UDF")

def add_YELLOW_total_amount_validation(dfrval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfrval = dfrval.withColumn('total_amount_val', udfYELLOWreceipt_val('total_amount','congestion_surcharge','extra',\
    'fare_amount','improvement_surcharge','mta_tax','tip_amount','tolls_amount'))

    return dfrval

YELLOW receipt validation testing function stored as UDF


In [80]:
def COLOR_datetime_validation(pudt,dodt,tdist):
# Function to validate datetime values
# Function returns 0 if datetimes together are valid, 1 if they are invalid, 2 if they do not exist
# We appear to need both for all queries with distance, so validation flags will be set together
# Please see the following comments for a more detailed explanation
    
    if pudt == "" or pudt == None or pudt =='nan' or dodt == "" or dodt == None or dodt =='nan':
        return 2
    
    # Checking overall length of date
    if len(pudt) == 19 and len(dodt) == 19:   
        # Convert to datetime to see if input follows the correct format
        try:
            pudt_DT = dt.strptime(pudt, '%Y-%m-%d %H:%M:%S')
            dodt_DT = dt.strptime(dodt, '%Y-%m-%d %H:%M:%S')
        except:
            return 1
    
        timeDelta = (dodt_DT - pudt_DT).total_seconds()
        # if the time difference is less than or equal to 0, we can return a 1
        if timeDelta <= 0:
            return 1

        # Otherwise, we can calculate our speed (in miles per second)

        # Try to validate speed (if distance exists and is itself valid)
        if tdist != None and tdist != "" and tdist != "nan":    
            try:
                tdist = float(tdist)
                tdist = abs(tdist)
                # Use only verifiable distances for now
                if tdist > 0 and tdist <= 300:
                    
                    speed = tdist/timeDelta*3600
                    # This will now be miles per hour
                    # Taxis could use freeway trips - 75mph is reasonable in come cases
                    if speed <= 75:
                        return 0
                    else:
                        return 1
                else:
                    return 0

            except:
            # We return 0 if nothing else is wrong but tdist is not verifiable or speed is incalculable
                return 0

        # We return 0 if nothing else is wrong but tdist is not verifiable or speed is incalculable
        else:
            return 0
    
    # If the date's length is invalid, return 1
    return 1

# Define function as a UDF for pyspark usage
udfCOLORdatetime_validation = udf(COLOR_datetime_validation, IntegerType())

print("Colors date testing function stored as UDF")

def add_GREEN_datetime_validation(dfDTval):
# Adds the datetime validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfDTval = dfDTval.withColumn('lpep_pickup_datetime_val', udfCOLORdatetime_validation('lpep_pickup_datetime',\
                                                                                        'lpep_dropoff_datetime',\
                                                                                        'trip_distance'))
    dfDTval = dfDTval.withColumn('lpep_dropoff_datetime_val', col('lpep_pickup_datetime_val'))

    return dfDTval

def add_YELLOW_datetime_validation(dfDTval):
# Adds the datetime validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfDTval = dfDTval.withColumn('tpep_pickup_datetime_val', udfCOLORdatetime_validation('tpep_pickup_datetime',\
                                                                                        'tpep_dropoff_datetime',\
                                                                                        'trip_distance'))
    dfDTval = dfDTval.withColumn('tpep_dropoff_datetime_val', col('tpep_pickup_datetime_val'))

    return dfDTval

Colors date testing function stored as UDF


In [81]:
# We want to check if each row can be used in our query - the full row, not just a subset
# We here test that all of our key variables are valid

def COLOR_valid_row(pickup_datetime_val,\
                     dropoff_datetime_val,\
                     pulocationid_val,\
                     dolocationid_val,\
                     trip_distance_val,\
                     tip_amount_val,\
                     total_amount_val,\
                     payment_type_val):
    
    if pickup_datetime_val == 1\
    or dropoff_datetime_val== 1\
    or pulocationid_val == 1\
    or dolocationid_val == 1\
    or payment_type_val == 1\
    or trip_distance_val == 1\
    or tip_amount_val==1\
    or total_amount_val == 1:
        
        return 1
    
    elif pickup_datetime_val == 2\
    or dropoff_datetime_val == 2\
    or pulocationid_val == 2\
    or dolocationid_val == 2\
    or payment_type_val == 2\
    or trip_distance_val == 2\
    or tip_amount_val == 2\
    or total_amount_val == 2:
        
        return 2
    
    return 0


# Define function as a UDF for pyspark usage
udfCOLOR_valid_row = udf(COLOR_valid_row, IntegerType())

print("Green and Yellow row testing function stored as UDF")

def add_GREEN_row_validation(dfCOLORval):
# Adds the row validation as a new dataframe column, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications

    dfCOLORval = dfCOLORval.withColumn('row_val', udfCOLOR_valid_row('lpep_pickup_datetime_val',\
                                                                 'lpep_dropoff_datetime_val',\
                                                                 'pulocationid_val',\
                                                                 'dolocationid_val',\
                                                                 'trip_distance_val',\
                                                                 'tip_amount_val',\
                                                                 'total_amount_val',\
                                                                 'payment_type_val'))

    return dfCOLORval

def add_YELLOW_row_validation(dfCOLORval):
# Adds the row validation as a new dataframe column, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications

    dfCOLORval = dfCOLORval.withColumn('row_val', udfCOLOR_valid_row('tpep_pickup_datetime_val',\
                                                                 'tpep_dropoff_datetime_val',\
                                                                 'pulocationid_val',\
                                                                 'dolocationid_val',\
                                                                 'trip_distance_val',\
                                                                 'tip_amount_val',\
                                                                 'total_amount_val',\
                                                                 'payment_type_val'))

    return dfCOLORval

Green and Yellow row testing function stored as UDF


Again - what do we really need for our queries at a bare minimum - what can form the basis of our clean dataset?
For the queries:  
1. The monthly total number of trips: nothing
2. Monthly total number of trips in Manhattan and Brooklyn: pu&dolocationid
3. The monthly total receipts - exclude tips, but include fares, surcharges, taxes and tolls: total_amount & tip_amount
4. Average trip receipt: same - total_amount & tip_amount
5. Average cost per in-progress minute: pickup & dropoff datetime, total_amount & tip_amount
6. Average tip per trip: tip_amount, payment type
7. Median monthly average trip speed per borough: trip_distance & pickup & dropoff datetime
8. How long does it take to go to the airport?L pu&dolocationid & pickup & dropoff datetime

If a record has all of these values and meets our validity constraints, we can send it immediately to the clean data.  

For the cost related queries, it is specified that Cost should exclude tips, but include fares, surcharges, taxes and tolls. The total_amount, as specified by the dictionary, less the tip amount, will satisfy this condition, and so we will prioritise taking that variable in a complete form, and potentially repairing it with the others if it is missing. This has the added benefit that the total amount is already the most likely to be accurate, as it would need to be correct for customer billing purposes.

## 3. Cleaning the GREEN dataset

### Analysis of valid values and validity constraints

vendorid: value is an int either 1 or 2  
passenger_count: an int (above 10 is suspicious) 
ratecodeid: an int between 1 and 6  
store_and_fwd_flag: string "Y" or "N"  
payment_type: an int between 1 and 6 - if the payment_type is 2 while tip != 0, this is likely incorrect (we will trust the tip here)  
fare_amount: a float  
extra: a float either 0, 0.50, 1.00 
mta_tax: a float either 0 or 0.50  
improvement_surcharge: a float 0.30 ONLY 2015 ONWARDS  
tolls_amount: a float  
trip_type: int 1 or 2  
ehail_fee: does not appear in data dictionary!!! appears to be blank  
congestion_surcharge: does not appear in data dictionary!!! seems to need 2.75 or 0

lpep_pickup_datetime: 2020-06-24 04:38:00  - this and the dropoff datetime should not be 0, or have a negative delta
lpep_dropoff_datetime: 2020-06-24 04:38:00  
trip_distance: a float, >0 a trip with a distance of 0 is not, by definition, a trip  
pulocationid: int 1 to 263  
dolocationid: int 1 to 263 but can be imputed from the ratecodeid in very specific instances
tip_amount: a float, >0 we don't want to add further validity constraints here because you never know when a youtuber will get into a new york taxi looking for content. We have flagged it as worthy of investigating if it is >1000  
total_amount: a float >0 and must approximately equal the sum of the other values (a >1% difference between the two figires will be flagged for investigation) - can be imputed from those values if missing  

The ehail_fee does not appear in the data dictionary, but it appears to be blank in all cases. The congestion_surcharge also does not appear in the dictionary, but when it takes a value in the data it appears to be 2.75 in Green and 2.5 in Yellow. We propose to add conditions covering these two variables regardless - they appear unimportant, but could play a role in the imputation of total_amount if necessary, and excessive/obviously incorrect values should be avoided.  

In the following, we will class a record (not a value) as being dirty if it is unusable for our purposes in 2.4.

If the record itself is dirty, we will attempt to use the remaining variables to repair it, but otherwise we will only track errors caught by our constraints, and not repair them.

### Identifying and splitting off dirty records

In [82]:
clean_GREEN_directory = create_folder("cleaned Green", save_directory)
with open(v_direc + "clean_GREEN_directory",'wb') as cleanedGreendirec:
    pickle.dump(clean_GREEN_directory,cleanedGreendirec)
    
dirty_GREEN_directory = create_folder("dirty Green", save_directory)
with open(v_direc + "dirty_GREEN_directory",'wb') as dirtyGreendirec:
    pickle.dump(dirty_GREEN_directory,dirtyGreendirec)

folder already exists at /home/epb123/output/cleaned Green
folder already exists at /home/epb123/output/dirty Green


In [83]:
def green_cleanup():
# takes no arguments - splits Green into clean and dirty data
# returns a pyspark dataframe with dirty data

    dropped_duplicates_count = 0
    clean_count = 0
    row_count = 0
    
    dirtyLst = []
    missingLst = []
            
    for i in green_integratedPaths:
        dfi = create_df(i)
        dfi_row_count = dfi.count()
        row_count = row_count + dfi_row_count

        # Adding validation columns
        dfi = add_location_validation(dfi) 
        dfi = add_GREEN_datetime_validation(dfi)
        dfi = add_vendorid_validation(dfi)
        dfi = add_improvement_surcharge_validation(dfi,i)
        dfi = add_mta_tax_validation(dfi)
        dfi = add_extra_validation(dfi)
        dfi = add_fare_amount_validation(dfi)
        dfi = add_tip_amount_validation(dfi) 
        dfi = add_tolls_amount_validation(dfi) 
        dfi = add_GREEN_total_amount_validation(dfi)
        dfi = add_store_and_forward_flag_validation(dfi)
        dfi = add_trip_distance_validation(dfi)
        dfi = add_passenger_count_validation(dfi)
        dfi = add_trip_type_validation(dfi)
        dfi = add_payment_type_validation(dfi)
        dfi = add_ratecodeID_validation(dfi)
        dfi = add_ehail_validation(dfi)
        dfi = add_GREEN_congestion_surcharge_validation(dfi)
        dfi = add_GREEN_row_validation(dfi)

        # Store records which are not dirty (this does not consider variables useless for 2.4 completion)
        # We also want to keep records which we can use - namely those with a pair of locationID values
        dfiClean = dfi.filter((dfi.row_val == 0))
        
        dfiClean = dfiClean.drop(*['pulocationid_val',\
                                   'dolocationid_val',\
                                   'lpep_pickup_datetime_val',\
                                   'lpep_dropoff_datetime_val',\
                                   'vendorid_val',\
                                   'improvement_surcharge_val',\
                                   'mta_tax_val',\
                                   'extra_val',\
                                   'fare_amount_val',\
                                   'tip_amount_val',\
                                   'tolls_amount_val',\
                                   'total_amount_val',\
                                   'store_and_fwd_flag_val',\
                                   'trip_distance_val',\
                                   'passenger_count_val',\
                                   'trip_type_val',\
                                   'payment_type_val',\
                                   'ratecodeid_val',\
                                   'ehail_fee_val',\
                                   'congestion_surcharge_val',\
                                   'row_val'])

        # We can drop duplicates in the clean dataset here - they will all have pickup and dropoff times or locations
        # The times are stored to the millisecond and PySpark's drop_duplicates compares all columns
        # If two records have pickup and dropoff times identical to the millisecond, we can safely drop them
        # If two records have both location ids identical and identical dispatch numbers, we can also safely drop them
        duplicates_result = drop_dupes(dfiClean)
        dfiClean = duplicates_result[0]

        dropped_duplicates_count = dropped_duplicates_count + duplicates_result[1]

        dfiClean_row_count = duplicates_result[2]
        clean_count = clean_count + dfiClean_row_count

        # Save the cleaned dataframes for repairing later
        save_df(dfiClean,ntpath.basename(i) + 'clean', clean_GREEN_directory)
        
        dirtyInI = dfi.filter((dfi.row_val == 1) | (dfi.row_val == 2))\
            .withColumn("filename", lit(ntpath.basename(i)))
        
        # We then add the dirty records to a spark dataframe, with an additional column specifying file of origin
        if i == green_integratedPaths[0]:
            dfiDirty = dirtyInI

        else:
            dfiDirty = dfiDirty.unionByName(dirtyInI)
        
        dirtyInI = dirtyInI.withColumn("filename",col('filename').cast("string"))
        save_df(dirtyInI,ntpath.basename(i) + 'dirty', dirty_GREEN_directory)
        
        print(str(dfiClean_row_count) + ' clean records preserved out of ' + str(dfi_row_count) + \
              ' original records - ' + str(dfiClean_row_count/dfi_row_count*100)[0:4] + "%")
        print(str(dirtyInI.filter(dirtyInI.row_val == 1).count()) + " invalid records have been found")
        print(str(dirtyInI.filter(dirtyInI.row_val == 2).count()) + " records with missing data have been found")
        
        # We can use Pandas to keep track of what's going on - this really slows down the code, but statistics are nice.
        # Note: if you comment this out to make the code run faster, do not forget to delete the return
        
        b = dfi.filter(col('lpep_dropoff_datetime_val')==2).count()
        c = dfi.filter(col('lpep_pickup_datetime_val')==2).count()
        d = dfi.filter(col('dolocationid_val')==2).count()
        e = dfi.filter(col('pulocationid_val')==2).count()
        f = dfi.filter(col('vendorid_val')==2).count()
        g = dfi.filter(col('ratecodeid_val')==2).count()
        h = dfi.filter(col('payment_type_val')==2).count()
        j = dfi.filter(col('store_and_fwd_flag_val')==2).count()
        k = dfi.filter(col('trip_distance_val')==2).count()
        l = dfi.filter(col('passenger_count_val')==2).count()
        m = dfi.filter(col('tip_amount_val')==2).count()
        n = dfi.filter(col('fare_amount_val')==2).count()
        o = dfi.filter(col('total_amount_val')==2).count()
        p = dfi.filter(col('improvement_surcharge_val')==2).count()
        q = dfi.filter(col('extra_val')==2).count()
        r = dfi.count()
        s = dfi.filter(col('mta_tax_val')==2).count()
        t = dfi.filter(col('trip_type_val')==2).count()
        
        u = dfi.filter(col('tolls_amount_val')==2).count()
        v = dfi.filter(col('ehail_fee_val')==2).count()
        w = dfi.filter(col('congestion_surcharge_val')==2).count()
        x = dfi.filter(col('row_val')==2).count()
        
        missingLst.append([i,r,b,c,d,e,f,g,h,j,k,l,m,n,o,p,q,s,t,u,v,w,x])
        
        b1 = dfi.filter(col('lpep_dropoff_datetime_val')==1).count()
        c1 = dfi.filter(col('lpep_pickup_datetime_val')==1).count()
        d1 = dfi.filter(col('dolocationid_val')==1).count()
        e1 = dfi.filter(col('pulocationid_val')==1).count()
        f1 = dfi.filter(col('vendorid_val')==1).count()
        g1 = dfi.filter(col('ratecodeid_val')==1).count()
        h1 = dfi.filter(col('payment_type_val')==1).count()
        j1 = dfi.filter(col('store_and_fwd_flag_val')==1).count()
        k1 = dfi.filter(col('trip_distance_val')==1).count()
        l1 = dfi.filter(col('passenger_count_val')==1).count()
        m1 = dfi.filter(col('tip_amount_val')==1).count()
        n1 = dfi.filter(col('fare_amount_val')==1).count()
        o1 = dfi.filter(col('total_amount_val')==1).count()
        p1 = dfi.filter(col('improvement_surcharge_val')==1).count()
        q1 = dfi.filter(col('extra_val')==1).count()
        r1 = dfi.count()
        s1 = dfi.filter(col('mta_tax_val')==1).count()
        t1 = dfi.filter(col('trip_type_val')==1).count()
        
        u1 = dfi.filter(col('tolls_amount_val')==1).count()
        v1 = dfi.filter(col('ehail_fee_val')==1).count()
        w1 = dfi.filter(col('congestion_surcharge_val')==1).count()
        x1 = dfi.filter(col('row_val')==1).count()
        
        dirtyLst.append([i,r1,b1,c1,d1,e1,f1,g1,h1,j1,k1,l1,m1,n1,o1,p1,q1,s1,t1,u1,v1,w1,x1])

    PdfMissing=pd.DataFrame(missingLst,columns=['file','number of lines','lpep_dropoff_datetime_val','lpep_pickup_datetime_val',\
                                 'dolocationid_val','pulocationid_val','vendorid_val','ratecodeid_val',\
                                 'payment_type_val','store_and_fwd_flag_val','trip_distance_val',\
                                 'passenger_count_val', 'tip_amount_val','fare_amount_val','total_amount_val',\
                                 'improvement_surcharge_val','extra_val','mta_tax_val','trip_type_val','tolls_amount_val',\
                                 'ehail_fee_val','congestion_surcharge_val','row_val'])
        
    #Only the columns with at least one value different from zero are kept.
    PdfMissing = PdfMissing.loc[:, (PdfMissing != 0).any(axis=0)] 

    PdfDirty=pd.DataFrame(dirtyLst,columns=['file','number of lines','lpep_dropoff_datetime_val','lpep_pickup_datetime_val',\
                             'dolocationid_val','pulocationid_val','vendorid_val','ratecodeid_val',\
                             'payment_type_val','store_and_fwd_flag_val','trip_distance_val',\
                             'passenger_count_val', 'tip_amount_val','fare_amount_val','total_amount_val',\
                             'improvement_surcharge_val','extra_val','mta_tax_val','trip_type_val','tolls_amount_val',\
                             'ehail_fee_val','congestion_surcharge_val','row_val'])

    #Only the columns with at least one value different from zero are kept.
    PdfDirty = PdfDirty.loc[:, (PdfDirty != 0).any(axis=0)] 
        
    print("Out of " + str(row_count) + " original records, " + str(clean_count) + " records were already clean enough to use in queries.")
    print(str(dropped_duplicates_count) + " records were dropped as duplicates.")
    print(str(row_count - clean_count - dropped_duplicates_count) \
          + " dirty records remain for resolution.")
    
    return dfiDirty, PdfMissing, PdfDirty

In [84]:
green_cleanup_return = green_cleanup()
dirtyGREEN = green_cleanup_return[0]
missingGreen = green_cleanup_return[1]
dirtyDetailedGreen = green_cleanup_return[2]

opened /home/epb123/output/integrated Green/green_tripdata_2013-08.csv
0 duplicate entries removed
13 clean records preserved out of 15 original records - 86.6%
2 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated Green/green_tripdata_2013-09.csv
0 duplicate entries removed
89 clean records preserved out of 98 original records - 90.8%
9 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated Green/green_tripdata_2013-10.csv
0 duplicate entries removed
321 clean records preserved out of 338 original records - 94.9%
17 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated Green/green_tripdata_2013-11.csv
0 duplicate entries removed
729 clean records preserved out of 756 original records - 96.4%
27 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated 

In [85]:
# Clean the PySpark outputs from before
GREEN_cleanCSV_folderPath = cleanup(clean_GREEN_directory)
with open(v_direc + "GREEN_cleanCSV_folderPath",'wb') as cleanedGREENCSVdirec:
    pickle.dump(GREEN_cleanCSV_folderPath,cleanedGREENCSVdirec)

GREEN_dirtyCSV_folderPath = cleanup(dirty_GREEN_directory)
with open(v_direc + "GREEN_dirtyCSV_folderPath",'wb') as dirtyGREENCSVdirec:
    pickle.dump(GREEN_dirtyCSV_folderPath,dirtyGREENCSVdirec)

folder already exists at /home/epb123/output/cleaned Green/all CSV
folder already exists at /home/epb123/output/dirty Green/all CSV


Based on filtering for records with all of the information needed for the queries, we appear to already be at 5,177 dirty or missing of 154,014 original records. Let us examine them and see what can be repaired:

In [86]:
dirtyDetailedGreen

,file,number of lines,lpep_dropoff_datetime_val,lpep_pickup_datetime_val,dolocationid_val,pulocationid_val,ratecodeid_val,payment_type_val,trip_distance_val,passenger_count_val,tip_amount_val,fare_amount_val,total_amount_val,improvement_surcharge_val,extra_val,mta_tax_val,tolls_amount_val,congestion_surcharge_val,row_val
0,/home/epb123/output/integrated Green/green_tri...,15,0,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,2
1,/home/epb123/output/integrated Green/green_tri...,98,0,0,2,3,0,0,8,0,0,0,0,0,0,0,0,0,9
2,/home/epb123/output/integrated Green/green_tri...,338,1,1,4,2,0,0,13,0,0,0,0,0,1,0,0,0,17
3,/home/epb123/output/integrated Green/green_tri...,756,3,3,8,3,0,3,18,0,0,0,0,0,3,0,0,0,27
4,/home/epb123/output/integrated Green/green_tri...,1189,2,2,8,2,0,2,18,0,0,0,0,0,0,0,0,0,29
5,/home/epb123/output/integrated Green/green_tri...,1586,3,3,6,4,0,5,49,1,0,0,0,0,0,0,0,0,59
6,/home/epb123/output/integrated Green/green_tri...,1987,6,6,13,2,0,4,37,0,0,0,0,0,0,0,0,0,53
7,/home/epb123/output/integrated Green/green_tri...,2568,7,7,11,4,0,8,54,0,0,0,0,0,0,0,0,0,79
8,/home/epb123/output/integrated Green/green_tri...,2596,14,14,14,10,0,7,36,0,0,0,0,0,0,0,0,0,68
9,/home/epb123/output/integrated Green/green_tri...,2822,12,12,11,7,0,5,46,0,0,0,1,0,0,0,0,0,68


In [87]:
missingGreen

,file,number of lines,vendorid_val,ratecodeid_val,payment_type_val,store_and_fwd_flag_val,passenger_count_val,trip_type_val,congestion_surcharge_val,row_val
0,/home/epb123/output/integrated Green/green_tri...,15,0,0,0,0,0,15,15,0
1,/home/epb123/output/integrated Green/green_tri...,98,0,0,0,0,0,98,98,0
2,/home/epb123/output/integrated Green/green_tri...,338,0,0,0,0,0,306,338,0
3,/home/epb123/output/integrated Green/green_tri...,756,0,0,0,0,0,756,756,0
4,/home/epb123/output/integrated Green/green_tri...,1189,0,0,0,0,0,1189,1189,0
5,/home/epb123/output/integrated Green/green_tri...,1586,0,0,0,0,0,1585,1586,0
6,/home/epb123/output/integrated Green/green_tri...,1987,0,0,0,0,0,1230,1987,0
7,/home/epb123/output/integrated Green/green_tri...,2568,0,0,0,0,0,543,2568,0
8,/home/epb123/output/integrated Green/green_tri...,2596,0,0,0,0,0,167,2596,0
9,/home/epb123/output/integrated Green/green_tri...,2822,0,0,0,0,0,0,2822,0


In [88]:
# Store the dirty dataframe separately to avoid unfortunate errors
# We can update our cleaned values as we go

recleanedGREEN = dirtyGREEN

Out of our core variables, what is broken?

In [89]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.row_val == 1).count())+" rows containing dirty values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.row_val == 2).count())+" rows not containing dirty values but containing missing values")

We have 4687 rows containing dirty values and...
490 rows not containing dirty values but containing missing values


Which is very interesting. Looking at the summary, we can see that out of what is important, we still have quite a good set:  
- pickup_datetime: clean, not missing
- dropoff_datetime: clean, not missing
- pulocationid: dirty, not missing
- dolocationid: dirty, not missing
- trip_distance: dirty, not missing
- tip_amount: dirty, not missing
- total_amount: dirty, not missing
- payment_type: dirty, missing

We can continue with **total_amount**

In [90]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.total_amount_val == 1).count())+" dirty values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.total_amount_val == 2).count())+" missing values")

We have 1378 dirty values and...
0 missing values


In [91]:
# We can then analyse the dirty values - we know the 5% criterion will have caused many records to drop
# Let us first check to see if the values are incorrect strings
# We know the total_amounts here are not null, thus we can do...

dirtyGREENanalysis = dirtyGREEN.filter(dirtyGREEN.total_amount_val == 1)
print("we have " + str(dirtyGREENanalysis.select("total_amount",sql.col("total_amount").cast("float").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [92]:
# We can then continue with negatives

dirtyGREENanalysis = dirtyGREENanalysis.withColumn('total_amount',col('total_amount').cast(FloatType()))
dirtyGREENanalysis = dirtyGREENanalysis.withColumn('negative_total', sql.when(col('total_amount') < 0.0, 1).otherwise(0))
print("we appear to have " + str(dirtyGREENanalysis.filter(col('negative_total') == 1).count()) + " negative total_amount entries")

we appear to have 255 negative total_amount entries


In [93]:
# Let us now check to see if any of the records tripped the 1000 upper bound

print("there is "+str(dirtyGREENanalysis.filter(dirtyGREENanalysis.total_amount > 1000).count())+" lucky taxi driver")
dirtyGREENanalysis.filter(dirtyGREENanalysis.total_amount > 1000).select([col('filename'),col('total_amount')]).show()

there is 1 lucky taxi driver
+--------------------+------------+
|            filename|total_amount|
+--------------------+------------+
|green_tripdata_20...|      1202.3|
+--------------------+------------+



All other conditions were checked, so we must have approx. 1,122 total_amount records which are more than 5% away from the sum of their components. Given that the total_amount record should be the most accurate of the monetary records (as it is what would be actually received in the company accounts), we have a verification issue. It is of course possible that the record was mistakenly entered, but we should theoretically trust it over the others. The UDF to flag mistaken total_amount values also depends on non-repaired entries for the other cash-based variables, however it does correct for what would be common non-direct value related errors (string storage and negativity). We, in the end, choose to set the flagged total_amount values to null,  as the basic checking and the 5% margin of error should correct for small mistakes in the other variables while capturing large errors which would cast doubt over the record's integrity. 

Negative values can be then converted into positive ones, under the assumption that they are mistakenly entered. If we then remove duplicates after the re-addition to the clean records, we can cover ourselves in case the negative values are some sort of accounting reversal.  

For the lucky taxi driver, given that the total amount is within reason (i.e. the youtuber in New York), we can keep the record.

In [94]:
# Let's make the updates to the recleanedGREEN, which we will later inject back into the dataset.

# We have no strings to update - we can cast the column as a float
recleanedGREEN = recleanedGREEN.withColumn('total_amount',col('total_amount').cast(FloatType()))
# So first update values as absolute - probably faster to abs() the whole column than rely on conditionals
recleanedGREEN = recleanedGREEN.withColumn('total_amount', sql.abs(col('total_amount')))
# We accept the discovered outlier as probably correct
# Let us set the remainder of records = Null

def GREENtotalupdate(total,cong_s,extra,fare,improv,tax,tip,toll,ehail):
# Function to transform total_amount to null if conditions not held
# Function returns Null if it is invalid, total if it is valid
    
    #no need for this as we know it's not missing
#     if total == None or total == "" or total == 'nan':
#         return 2
#     try:
#         total = float(total)
#     except:
#         return 1 
    # No need for this as already done
    #difference = abs(total)
    difference = total
    for i in [cong_s,extra,fare,improv,tax,tip,toll,ehail]:
        if i == None or i == "" or i == 'nan' or i == 'NaN':
            i = 0
        try:
            i = abs(float(i))
        # Note we use abs here so the fact that we haven't checked tip yet does not affect it
            difference = difference - i
        except:
        # We do the same for float values - we simply want to ignore incorrect strings here
            pass
        
    if abs(difference) > total*0.05: #and total >= 0 and total <= 1000:
    # if the difference is greater than 5% of total, it is incorrect and we return None
        return None
    
    return total

# Define function as a UDF for pyspark usage
udfGREENtotalbroken = udf(GREENtotalupdate, FloatType())

print("Receipt updating testing function stored as UDF")

def add_GREEN_total_amount_update(dfrval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfrval = dfrval.withColumn('total_amount', udfGREENtotalbroken('total_amount','congestion_surcharge','extra',\
    'fare_amount','improvement_surcharge','mta_tax','tip_amount','tolls_amount','ehail_fee'))

    return dfrval

recleanedGREEN = add_GREEN_total_amount_update(recleanedGREEN)

# we only need to count once - we did not previously have missing values
print(str(recleanedGREEN.filter(col('total_amount').isNull()).count()) + " irreconcilable totals removed")

Receipt updating testing function stored as UDF
1122 irreconcilable totals removed


We can then move onto **tip_amount**

In [95]:
# Let's find out where any dropped records came from
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.tip_amount_val == 1).count())+" dirty values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.tip_amount_val == 2).count())+" missing values")

We have 3 dirty values and...
0 missing values


In [96]:
# Let us now check to see if the values are incorrect strings
# We know the tip_amounts here are not null, thus we can do...

dirtyGREENanalysis = dirtyGREEN.filter(dirtyGREEN.tip_amount_val == 1)
print("we have " + str(dirtyGREENanalysis.select("tip_amount",sql.col("tip_amount").cast("float").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [97]:
# We can then continue with negatives

dirtyGREENanalysis = dirtyGREENanalysis.withColumn('tip_amount',col('tip_amount').cast(FloatType()))
dirtyGREENanalysis = dirtyGREENanalysis.withColumn('negative_tip', sql.when(col('tip_amount') < 0.0, 1).otherwise(0))
print("we appear to have " + str(dirtyGREENanalysis.filter(col('negative_tip') == 1).count()) + " negative tip_amount entries")

we appear to have 3 negative tip_amount entries


Well - as before, those three tip_amount entries can be repaired and put back into the dataset

In [98]:
# Let's make the updates to the recleanedGREEN, which we will later inject back into the dataset.

# We have no strings to update - we can cast the column as a float
recleanedGREEN = recleanedGREEN.withColumn('tip_amount',col('tip_amount').cast(FloatType()))
# So update values as absolute - probably faster to abs() the whole column than rely on conditionals
recleanedGREEN = recleanedGREEN.withColumn('tip_amount', sql.abs(col('tip_amount')))

Let's start on **payment_type**

In [99]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.payment_type_val == 1).count())+" dirty values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.payment_type_val == 2).count())+" missing values")

We have 589 dirty values and...
512 missing values


We can impute missing values - we have just fixed the tip amount  
- If there is a Null value but tip exists, we store as card  
- If there is a Null value but tip == 0, we store as cash  

We assume the tip would not exist with a void trip or a no charge trip.
We also make the assumption that no tip strongly correlates with a cash trip (in a country where restaurant tips are 20% standard, we believe this is reasonable)

In [100]:
# We can impute missing values - we have just fixed the tip amount
# If there is a Null value but tip exists, we store as card
# If there is a Null value but tip == 0, we store as cash
# We assume the tip would not exist with a void trip or a no charge trip
dirtyGREENanalysis = dirtyGREEN
print("we have " + str(dirtyGREENanalysis.filter((col("payment_type").isNull()) & (col("tip_amount") != 0)).count())\
                       + " trips with a tip but no payment type stored")
print("we have " + str(dirtyGREENanalysis.filter((col("payment_type").isNull()) & (col("tip_amount") == 0)).count())\
                       + " trips with no tip and no payment type stored")

we have 72 trips with a tip but no payment type stored
we have 440 trips with no tip and no payment type stored


In [101]:
recleanedGREEN = recleanedGREEN.withColumn('payment_type',sql.when((col("payment_type").isNull())\
                                                                   & (col("tip_amount") != 0), '1')\
                                           .otherwise(col('payment_type')))
recleanedGREEN = recleanedGREEN.withColumn('payment_type',sql.when((col("payment_type").isNull())\
                                                                   & (col("tip_amount") == 0), '2')\
                                           .otherwise(col('payment_type')))

In [102]:
print("we have " + str(recleanedGREEN.filter((col("payment_type").isNull()) & (col("tip_amount") != 0)).count())\
                       + " remaining trips with a tip but no payment type stored")
print("we have " + str(recleanedGREEN.filter((col("payment_type").isNull()) & (col("tip_amount") == 0)).count())\
                       + " remaining trips with no tip and no payment type stored")

we have 0 remaining trips with a tip but no payment type stored
we have 0 remaining trips with no tip and no payment type stored


In [103]:
dirtyGREENanalysis = dirtyGREEN.filter(dirtyGREEN.payment_type_val == 1)

# We can test invalid payment types
# Let us first check to see if the values are incorrect strings
# We know the payment_types here are not null, thus we can do...

print("we have " + str(dirtyGREENanalysis.filter(col("payment_type").cast("int").isNull()).count())\
                       + " incorrectly stored strings")

we have 0 incorrectly stored strings


Now that the strings are dealt with, we can test valid entries

In [104]:
print("We have " + str(dirtyGREENanalysis.filter(col("payment_type").cast("int") > 6).count())\
                       + " values above 6")
print("We have " + str(dirtyGREENanalysis.filter(col("payment_type").cast("int") < 1).count())\
                       + " values below 1")

We have 0 values above 6
We have 0 values below 1


We should check how many trips were either 3 or 6:  
1) Credit card  
2) Cash  
3) No charge  
4) Dispute  
5) Unknown  
6) Voided trip  

We have flagged trips in 3 or 6 for removal because they are invalid - we do not want to analyse free trips or trips which did not exist, particularly with queries based on amounts. Trips in unknown, we cannot classify, and just because someone decided to dispute their fare did not mean by itself that it was not valid.

In [105]:
print("We will need to drop "+str(dirtyGREENanalysis.filter(col('payment_type') == 3).count())\
      + " trips with no charge.")
print("We will need to drop "+str(dirtyGREENanalysis.filter(col('payment_type') == 6).count())\
      + " voided trips.")

We will need to drop 589 trips with no charge.
We will need to drop 0 voided trips.


In [106]:
recleanedGREEN = recleanedGREEN.filter(col('payment_type') != 3)
recleanedGREEN = recleanedGREEN.filter(col('payment_type') != 6)

In [107]:
print("We now have "+str(recleanedGREEN.filter(col('payment_type') == 3).count())\
      + " trips with no charge.")
print("We now have "+str(recleanedGREEN.filter(col('payment_type') == 6).count())\
      + " voided trips.")

We now have 0 trips with no charge.
We now have 0 voided trips.


It is interesting that there were no recorded tips in a cash flagged trip, but it is possible the cab driver needs to press the 'card' button to take a card payment, eliminating the error. As we will see later, we do not have the same behaviour from our Yellow drivers.

We can then move onto **trip_distance**

In [108]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.trip_distance_val == 1).count())+" dirty values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.trip_distance_val == 2).count())+" missing values")

We have 2167 dirty values and...
0 missing values


In [109]:
# Let us now check to see if the values are incorrect strings
# We know the trip_distances here are not null, thus we can do...

dirtyGREENanalysis = dirtyGREEN.filter(dirtyGREEN.trip_distance_val == 1)
print("we have " + str(dirtyGREENanalysis.select("trip_distance",sql.col("trip_distance").cast("float").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [110]:
# We can then continue with negatives

dirtyGREENanalysis = dirtyGREENanalysis.withColumn('trip_distance',col('trip_distance').cast(FloatType()))
dirtyGREENanalysis = dirtyGREENanalysis.withColumn('negative_trip', sql.when(col('trip_distance') < 0.0, 1).otherwise(0))
print("we appear to have " + str(dirtyGREENanalysis.filter(col('negative_trip') == 1).count()) + " negative trip_distance entries")

we appear to have 1 negative trip_distance entries


In [111]:
print("there are "+str(dirtyGREENanalysis.filter(dirtyGREENanalysis.trip_distance == 0).count())+" trips to nowhere")
#dirtyGREENanalysis.filter(dirtyGREENanalysis.trip_distance == 0).select([col('filename'),col('trip_distance')]).show()

there are 2166 trips to nowhere


In [112]:
# Last is to check trips with distance >= 300
print("there are "+str(dirtyGREENanalysis.filter(sql.abs(dirtyGREENanalysis.trip_distance) >= 300.0).count())+" trips to somewhere far far away")
dirtyGREENanalysis.filter(sql.abs(dirtyGREENanalysis.trip_distance) >= 300).select([col('filename'),col('trip_distance')]).show()

there are 0 trips to somewhere far far away
+--------+-------------+
|filename|trip_distance|
+--------+-------------+
+--------+-------------+



We have an explanation for our dirty values. The question is now to determine what was an actual trip with an incorrect distance and what was a "trip" which never moved. Both are possible - imagine getting in a taxi and then getting out at the next intersection because you forgot something - but we would be better off knowing which case was which. As a simple rule, we shall change the trip_distance to null if the distance is 0, but the total_amount is > 5 or the tip_amount is >0, as a 0 distance here is misleading. We will, however, keep trips with distances truly = 0 (based on the amount).

We originally only used the total_amount, but we have already cleaned it and have therefore lost some values above the margin of error. We can use tip_amount alongside it - if a fare pays a tip, the distance could not possibly be 0 (although it is New York, this is still highly unlikely).

In [113]:
# Let's make the updates to the recleanedGREEN, which we will later inject back into the dataset.

# let's first handle that negative value
# Abs-ing the entire column is a little bit inelegant, but will suffice

# We have no strings to update - we can cast the column as a float
recleanedGREEN = recleanedGREEN.withColumn('trip_distance',col('trip_distance').cast(FloatType()))
# So first update values as absolute - probably faster to abs() the whole column than rely on conditionals
recleanedGREEN = recleanedGREEN.withColumn('trip_distance', sql.abs(col('trip_distance')))

# Count the dirty records - use the pre-filtered dataset here
print("we have "+str(recleanedGREEN.filter(( col('trip_distance') == 0.0)\
            & ( (col('total_amount') >= 5.0) | (col("tip_amount") > 0.0))).count())+\
            " trips with 0 distances and a nonzero tip or an amount > $5")

# Next let's apply our filtering criteria to those trip_distances
recleanedGREEN = recleanedGREEN.withColumn('trip_distance', sql.when( ( col('trip_distance') == 0.0)\
                                            & ( (col('total_amount') >= 5.0) | (col("tip_amount") > 0.0)),\
                                            None).otherwise(col('trip_distance')))

print("we have "+str(recleanedGREEN.filter(( col('trip_distance') == 0.0)\
            & ( (col('total_amount') >= 5.0) | (col("tip_amount") > 0.0))).count())+\
            " expensive trips with 0 distances and a nonzero tip or an amount > $5 remaining")

we have 1314 trips with 0 distances and a nonzero tip or an amount > $5
we have 0 expensive trips with 0 distances and a nonzero tip or an amount > $5 remaining


And the rest we will keep.

We can next analyse the **dolocationid** and **pulocationid**

In [114]:
# Let's find out where our dropped records came from
print("We have "+str(recleanedGREEN.filter(recleanedGREEN.dolocationid_val == 1).count())+" dirty dropoff values and...")
print(str(recleanedGREEN.filter(dirtyGREEN.dolocationid_val == 2).count())+" missing dropoff values")
print("We have "+str(recleanedGREEN.filter(recleanedGREEN.pulocationid_val == 1).count())+" dirty pickup values and...")
print(str(recleanedGREEN.filter(recleanedGREEN.pulocationid_val == 2).count())+" missing pickup values")

We have 592 dirty dropoff values and...
0 missing dropoff values
We have 320 dirty pickup values and...
0 missing pickup values


Let's then start with **pulocationid**

In [115]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...
# We need to use recleanedGREEN because we have dropped some records already

dirtyGREENanalysis = recleanedGREEN.filter(col('pulocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyGREENanalysis.filter(col("pulocationid").cast("int").isNull()).count())\
      + " incorrectly stored strings")

we have 208 incorrectly stored strings


These would result from a "failed to match" failure coming out of the 2.2 locationid calculations. Basically, if the coordinates were corrupt - i.e. something was stored at 5 deg. longitude and 36 deg latitude, the shapefile would not identify the location as being in New York. We can set these equal to Null to avoid any later confusion.

In [116]:
# Testing the theory...
print("we have "+str(recleanedGREEN.filter(col('pulocationid') == "failed to match").count())+" match failures")

# Next let's apply our filtering criteria to those locationids
recleanedGREEN = recleanedGREEN.withColumn('pulocationid', sql.when(col('pulocationid') == "failed to match",\
                                            None).otherwise(col('pulocationid')))

print("we have re-valued "+str(recleanedGREEN.filter(col('pulocationid').isNull()).count())+" match failures")

we have 208 match failures
we have re-valued 208 match failures


We will have run the locationid testing code in 2.2, and the resulting errors must therefore come from values 264, 265, and above.

In [117]:
print("We have "+str(recleanedGREEN.filter(col('pulocationid') > 263).count())+" values above our threshold")

We have 112 values above our threshold


In [118]:
recleanedGREEN = recleanedGREEN.withColumn('pulocationid', sql.when(col('pulocationid') > 263,\
                                            None).otherwise(col('pulocationid')))
print(str(recleanedGREEN.filter(col('pulocationid') > 263).count())+" values remain after cleaning")

0 values remain after cleaning


dolocationid appears to have the same type of error.

In [119]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...
# Need to use recleanedGREEN here!!!

dirtyGREENanalysis = recleanedGREEN.filter(col('dolocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyGREENanalysis.filter(col("dolocationid").cast("int").isNull()).count()) + " incorrectly stored strings")

we have 390 incorrectly stored strings


In [120]:
# Testing the theory...
print("we have "+str(recleanedGREEN.filter(col('dolocationid') == "failed to match").count())+" match failures")

# Next let's apply our filtering criteria to those locationids
recleanedGREEN = recleanedGREEN.withColumn('dolocationid', sql.when(col('dolocationid') == "failed to match",\
                                            None).otherwise(col('dolocationid')))

print("we have re-valued "+str(recleanedGREEN.filter(col('dolocationid').isNull()).count())+" match failures")

we have 390 match failures
we have re-valued 390 match failures


In [121]:
print("We have "+str(recleanedGREEN.filter(col('dolocationid') > 263).count())+" values above our threshold")

We have 202 values above our threshold


In [122]:
recleanedGREEN = recleanedGREEN.withColumn('dolocationid', sql.when(col('dolocationid') > 263,\
                                            None).otherwise(col('dolocationid')))

print(str(recleanedGREEN.filter(col('dolocationid') > 263).count())+" values remain above the threshold")

0 values remain above the threshold


Let's not forget to examine the incorrect **lpep_pickup/dropoff_datetime** columns

In [123]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.lpep_dropoff_datetime_val == 1).count())+" dirty dropoff values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.lpep_dropoff_datetime_val == 2).count())+" missing dropoff values")
print("We have "+str(dirtyGREEN.filter(dirtyGREEN.lpep_pickup_datetime_val == 1).count())+" dirty pickup values and...")
print(str(dirtyGREEN.filter(dirtyGREEN.lpep_pickup_datetime_val == 2).count())+" missing pickup values")

We have 450 dirty dropoff values and...
0 missing dropoff values
We have 450 dirty pickup values and...
0 missing pickup values


In [124]:
del dirtyGREEN
del dirtyGREENanalysis
# We are no longer using these, and will need the memory

Given the prior issues with trip_distance, we can be reasonably confident that there will be further mismatches. Any trip with an average speed >75mph in central NYC will be an incorrect record - while the several long journeys we have would allow us to assume someone might average 55 (speed limit in NY) or so on the highway, an average above 75 when so much time would be lost moving through NYC would be almost impossible. We will therefore first correct for any trips demonstrating this error - they will almost certainly be incorrect, and so we will drop the records. 

Along with this, we can correct for invalid formats (by setting to Null), and afterwards we can examine dropoffs before pickups (where we will repair by switching pickuptime and dropoff time).

In [125]:
# We will here make a UDF to set null all excessive speed records (and all 0 time deltas and format errors while we are at it)

def COLOR_datetime_speed_update(pudt,dodt,tdist):
# Let us return a flag for whether or not to drop - True to delete       

    # If one column is already gone, we want to set both columns to 0 regardless as we are only interested in the difference
    if pudt == "" or pudt == None or pudt =='nan' or dodt == "" or dodt == None or dodt =='nan':
        return True
    
    # Tf the time does not follow a correct format, delete it
    try:
        pudt_DT = dt.strptime(pudt, '%Y-%m-%d %H:%M:%S')
        dodt_DT = dt.strptime(dodt, '%Y-%m-%d %H:%M:%S')
    except:
        return True
    
    # If we haven't hit a None yet, timeDelta will exist, but we have not yet switched values if less than 0
    timeDelta = (dodt_DT - pudt_DT).total_seconds()
    
    if timeDelta == 0:
    # we can drop if the difference in time, to the millisecond, is 0 - this was not a trip
        return True
    
    timeDelta = abs(timeDelta)

    # Try to validate speed (if distance exists and is itself valid)
    # We have already converted tdist, but in case we move the order around we still want this to be valid
    if tdist != None and tdist != "" and tdist != "nan":    
        try:
            tdist = float(tdist)
            tdist = abs(tdist)
            
            speed = tdist/timeDelta*3600
            # This will now be miles per hour
            # Taxis could use freeway trips - 75mph is reasonable in come cases
            if speed > 75:
                return True
            else:
                return False
            
        except:
        # We have already converted tdist, but if we move the order we want this to be valid
        # We have already returned if timeDelta == 0, so we do not need to catch the exception otherwise
            return False
    else:
    # if we have no distance, we don't want to drop
        return False

# Define function as a UDF for pyspark usage
udfCOLORdatetime_drop_flag = udf(COLOR_datetime_speed_update, BooleanType())

print("Colors drop flag function stored as UDF")

Colors drop flag function stored as UDF


In [126]:
# We will here make the UDF such that it is applicable to Colorful taxi types:
# If we have moved past our exclusion criteria, we want to see if we should swap the columns to repair the records

def COLOR_datetime_switch_flag(pudt,dodt):
# We have just set invalid timestamps to Null, so
    if pudt == "" or pudt == None or pudt =='nan' or dodt == "" or dodt == None or dodt =='nan':
    # If Null, we don't want to switch them anyway
        return 0
    
    pudt_DT = dt.strptime(pudt, '%Y-%m-%d %H:%M:%S')
    dodt_DT = dt.strptime(dodt, '%Y-%m-%d %H:%M:%S')
    
    timeDelta = (dodt_DT - pudt_DT).total_seconds()
    if timeDelta < 0:
        return 1
    else:
        return 0

# Define function as a UDF for pyspark usage
udfCOLORdatetime_switch = udf(COLOR_datetime_switch_flag, IntegerType())

print("Colors date testing function stored as UDF")

Colors date testing function stored as UDF


In [127]:
recleanedGREEN = recleanedGREEN.withColumn('dropdt', udfCOLORdatetime_drop_flag('lpep_pickup_datetime',\
                                                                                'lpep_dropoff_datetime',\
                                                                                'trip_distance'))
# Drop columns when datetime flag is True!
recleanedGREEN = recleanedGREEN.withColumn('lpep_dropoff_datetime',sql.when(col('dropdt') == True, None).\
                                           otherwise(col('lpep_dropoff_datetime')))
recleanedGREEN = recleanedGREEN.withColumn('lpep_pickup_datetime',sql.when(col('dropdt') == True, None).\
                                           otherwise(col('lpep_pickup_datetime')))

recleanedGREEN = recleanedGREEN.drop('dropdt')

In [128]:
# We can do this because we have no pre-existing Nulls

print("In the current version of the cleaned file, we now have... "+\
     str(recleanedGREEN.filter(recleanedGREEN.lpep_dropoff_datetime.isNull()).count())+\
     " fewer dropoff values and "+\
     str(recleanedGREEN.filter(recleanedGREEN.lpep_pickup_datetime.isNull()).count())+\
     " fewer pickup values.")

In the current version of the cleaned file, we now have... 373 fewer dropoff values and 373 fewer pickup values.


In [129]:
# Make the switch Flag dummy
recleanedGREEN = recleanedGREEN.withColumn('tsFlag',udfCOLORdatetime_switch('lpep_pickup_datetime','lpep_dropoff_datetime'))
print("We will now repair "+str(recleanedGREEN.filter(col('tsFlag') == 1).count())+" backwards values")

# Put dropoff values into a temporary column
recleanedGREEN = recleanedGREEN.withColumn('dropoff_temp',col('lpep_dropoff_datetime'))

# Move pickup values into dropoff when switch flag is 1
recleanedGREEN = recleanedGREEN.withColumn('lpep_dropoff_datetime',sql.when(col('tsFlag') == 1,col('lpep_pickup_datetime')).\
                                           otherwise(col('lpep_dropoff_datetime')))
# Move dropoff values from temporary column into pickup when switch flag is 1
recleanedGREEN = recleanedGREEN.withColumn('lpep_pickup_datetime',sql.when(col('tsFlag') == 1,col('dropoff_temp')).\
                                           otherwise(col('lpep_pickup_datetime')))

# Drop temporary columns
recleanedGREEN = recleanedGREEN.drop(*['tsFlag','dropoff_temp'])

We will now repair 1 backwards values


Making a total of 450 errors caught and 450 errors dealt with.

We have essentially isolated and fixed each category of error which our filters picked up. While we still have further errors in other variables, our main set and the set relevant for the query has been fully analysed. We have replaced incorrect and irreparable values with Null, and repaired those which we could. We can now re-merge our recleanedGREEN dataset with the other data.  

Let's start by checking how many complete (for 2.4 purposes) records we can extract:

In [130]:
# We've set irreparable values as Null, so...
print("We have managed to repair "+str(recleanedGREEN.filter(\
                                                             (col('lpep_pickup_datetime').isNotNull()) &\
                                                             (col('lpep_dropoff_datetime').isNotNull()) &\
                                                             (col('dolocationid').isNotNull()) &\
                                                             (col('pulocationid').isNotNull()) &\
                                                             (col('trip_distance').isNotNull()) &\
                                                             (col('total_amount').isNotNull()) &\
                                                             (col('tip_amount').isNotNull()) &
                                                             (col('payment_type').isNotNull())\
                                                             ).count())+" complete records")

We have managed to repair 1244 complete records


Given that this is approximately 24% out of all of the dirty records we discovered, to have 1244 complete and usable additions is an acceptable result. The question is then what we do with the remaining incomplete records. Given that this represents ~2% of the raw dataset, it would be nice to include them. However, we would then have a changing overall database size across queries. Our results would therefore be inconsistent across queries.  

We thus proceed by returning these fully cleaned values to their respective datasets, and storing the 3933 dirty ones.

In [131]:
# Let us filter for the values we want
everdirtyGREEN = recleanedGREEN.filter(\
                                     (col('lpep_pickup_datetime').isNull()) |\
                                     (col('lpep_dropoff_datetime').isNull()) |\
                                     (col('dolocationid').isNull()) |\
                                     (col('pulocationid').isNull()) |\
                                     (col('trip_distance').isNull()) |\
                                     (col('total_amount').isNull()) |\
                                     (col('tip_amount').isNull()) |\
                                     (col('payment_type').isNull()))

recleanedGREEN = recleanedGREEN.filter(\
                                     (col('lpep_pickup_datetime').isNotNull()) &\
                                     (col('lpep_dropoff_datetime').isNotNull()) &\
                                     (col('dolocationid').isNotNull()) &\
                                     (col('pulocationid').isNotNull()) &\
                                     (col('trip_distance').isNotNull()) &\
                                     (col('total_amount').isNotNull()) &\
                                     (col('tip_amount').isNotNull()) &\
                                     (col('payment_type').isNotNull()))

# Before re-integration, we must remember that we have re-cast a lot of columns as floats and the rest of the data is StringType
everdirtyGREEN = everdirtyGREEN.select([col(i).cast("string") for i in everdirtyGREEN.columns])
recleanedGREEN = recleanedGREEN.select([col(i).cast("string") for i in recleanedGREEN.columns])

# We can then drop the validation columns
recleanedGREEN = recleanedGREEN.drop(*['pulocationid_val',\
                                   'dolocationid_val',\
                                   'lpep_pickup_datetime_val',\
                                   'lpep_dropoff_datetime_val',\
                                   'vendorid_val',\
                                   'improvement_surcharge_val',\
                                   'mta_tax_val',\
                                   'extra_val',\
                                   'fare_amount_val',\
                                   'tip_amount_val',\
                                   'tolls_amount_val',\
                                   'total_amount_val',\
                                   'store_and_fwd_flag_val',\
                                   'trip_distance_val',\
                                   'passenger_count_val',\
                                   'trip_type_val',\
                                   'payment_type_val',\
                                   'ratecodeid_val',\
                                   'ehail_fee_val',\
                                   'congestion_surcharge_val',\
                                   'row_val'])

# leaving us with just the filename variable as something which differentiates the schema from our clean data

In [132]:
def GREEN_combine(cleanedupGREENDF,alwaysdirtyGREENDF):
    
    reAdded = 0
    repair_filepaths = [xxx.filename for xxx in recleanedGREEN.select('filename').distinct().collect()]

    for i in range(0,len(repair_filepaths)):
        repair_filepaths[i] = clean_GREEN_directory+"/all CSV/"+repair_filepaths[i]+"clean"
    
    repair_filepaths.sort()
    for i in repair_filepaths:
        tempFilterDF = recleanedGREEN.filter(col('filename') == i.replace(clean_GREEN_directory,"").\
                                                                      replace('/all CSV/',"").\
                                                                      replace('csvclean','csv'))
        dfi = create_df(i)
        pre_existing = dfi.count()

        # We do not repair duplicates here as it is too expensive for the cluster and catches 0 results
        
        if pre_existing == 0:
            dfi = tempFilterDF    
        else:
            dfi = dfi.unionByName(tempFilterDF.drop('filename'))
        addedDFI = dfi.count()-pre_existing
        print(str(addedDFI)+" additional records added")
        reAdded = reAdded + addedDFI
        save_df(dfi,ntpath.basename(i), clean_GREEN_directory)

    print(str(reAdded)+" records added across all files")
    
    if reAdded == 0:
        print("WARNING: REPAIRED RECORDS COULD NOT BE RE-ADDED - ORIGINAL FILE WAS EMPTY")
    
    # We have the dirty records saved on a monthly basis already anyway
    # We will therefore save everdirtyGREEN alone
    # We will then have saved files per month with caught dirty records and one master file with unrecoverable records
    # This will record unrecoverable values, but will not take hours to run
    # We do not coalesce() here as the file is large (and we will not be reading it again outside of spark)
    
    alwaysdirtyGREENDF.write.options(header = True).mode("overwrite").\
         csv(os.path.join(dirty_GREEN_directory, "irreparable_GREEN_records"))

In [133]:
GREEN_combine(recleanedGREEN, everdirtyGREEN)

opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2013-08.csvclean
1 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2013-09.csvclean
4 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2013-10.csvclean
8 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2013-11.csvclean
9 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2013-12.csvclean
5 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2014-01.csvclean
14 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2014-02.csvclean
17 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2014-03.csvclean
27 additional records added
opened /home/epb123/output/cleaned Green/all CSV/green_tripdata_2014-04.csvclean
9 additional records added
opened /home/epb123/outpu

In [134]:
# Clean the PySpark outputs from before
GREEN_cleanCSV_folderPath = cleanup(clean_GREEN_directory)
with open(v_direc + "GREEN_cleanCSV_folderPath",'wb') as cleanedGREENCSVdirec:
    pickle.dump(GREEN_cleanCSV_folderPath,cleanedGREENCSVdirec)

# GREEN_dirtyCSV_folderPath = cleanup(dirty_GREEN_directory)
# with open(v_direc + "GREEN_dirtyCSV_folderPath",'wb') as dirtyGREENCSVdirec:
#     pickle.dump(GREEN_dirtyCSV_folderPath,dirtyGREENCSVdirec)

folder already exists at /home/epb123/output/cleaned Green/all CSV


In [135]:
del recleanedGREEN
del everdirtyGREEN

## 4. Cleaning the YELLOW dataset

### Analysis of valid values and validity constraints

vendorid: value is an int either 1 or 2  
passenger_count: an int (above 10 is suspicious) 
ratecodeid: an int between 1 and 6  
store_and_fwd_flag: string "Y" or "N"  
payment_type: an int between 1 and 6  
fare_amount: a float  
extra: a float either 0, 0.50, 1.00 
mta_tax: a float either 0 or 0.50  
improvement_surcharge: a float 0.30 ONLY 2015 ONWARDS  
tolls_amount: a float  
congestion_surcharge: does not appear in data dictionary!!! seems to need 2.75 or 0

tpep_pickup_datetime: 2020-06-24 04:38:00  
tpep_dropoff_datetime: 2020-06-24 04:38:00  
trip_distance: a float, >0 a trip with a distance of 0 is not, by definition, a trip  
pulocationid: int 1 to 263  
dolocationid: int 1 to 263 but can be imputed from the ratecodeid  
tip_amount: a float, >0 we don't want to add further validity constraints here because you never know when a youtuber will get into a new york taxi looking for content. We have flagged it as worthy of investigating if it is >1000  
total_amount: a float >0 and must approximately equal the sum of the other values (a >1% difference between the two figires will be flagged for investigation) - can be imputed from those values if missing  

We have minor differences between the green and yellow datasets - namely the renaming of the lpep to tpep, and the absense of ehail and trip_type - but they are minor enough that we can use similar functions to those we developed for Green.

### Identifying and splitting off dirty records

In [136]:
clean_YELLOW_directory = create_folder("cleaned YELLOW", save_directory)
with open(v_direc + "clean_YELLOW_directory",'wb') as cleanedYELLOWdirec:
    pickle.dump(clean_YELLOW_directory,cleanedYELLOWdirec)
    
dirty_YELLOW_directory = create_folder("dirty YELLOW", save_directory)
with open(v_direc + "dirty_YELLOW_directory",'wb') as dirtyYELLOWdirec:
    pickle.dump(dirty_YELLOW_directory,dirtyYELLOWdirec)

folder already exists at /home/epb123/output/cleaned YELLOW
folder already exists at /home/epb123/output/dirty YELLOW


In [137]:
def yellow_cleanup():
# takes no arguments - splits Yellow into clean and dirty data
# returns a pyspark dataframe with dirty data

    dropped_duplicates_count = 0
    clean_count = 0
    row_count = 0
    
    dirtyLst = []
    missingLst = []
            
    for i in yellow_integratedPaths:
        dfi = create_df(i)
        dfi_row_count = dfi.count()
        row_count = row_count + dfi_row_count

        # Adding validation columns
        # Note - for YELLOW, the UDFs seem to be very slow.
        # In future, further spark native functions should be used
        dfi = add_location_validation(dfi) 
        dfi = add_YELLOW_datetime_validation(dfi)
        dfi = add_vendorid_validation(dfi)
        dfi = add_improvement_surcharge_validation(dfi,i)
        dfi = add_mta_tax_validation(dfi)
        dfi = add_extra_validation(dfi)
        dfi = add_fare_amount_validation(dfi)
        dfi = add_tip_amount_validation(dfi) 
        dfi = add_tolls_amount_validation(dfi) 
        dfi = add_YELLOW_total_amount_validation(dfi)
        dfi = add_store_and_forward_flag_validation(dfi)
        dfi = add_trip_distance_validation(dfi)
        dfi = add_passenger_count_validation(dfi)
        dfi = add_payment_type_validation(dfi)
        dfi = add_ratecodeID_validation(dfi)
        dfi = add_YELLOW_congestion_surcharge_validation(dfi)
        dfi = add_YELLOW_row_validation(dfi)

        # Store records which are not dirty (this does not consider variables useless for 2.4 completion)
        # We also want to keep records which we can use - namely those with a pair of locationID values
        dfiClean = dfi.filter((dfi.row_val == 0))
        
        dfiClean = dfiClean.drop(*['pulocationid_val',\
                                   'dolocationid_val',\
                                   'tpep_pickup_datetime_val',\
                                   'tpep_dropoff_datetime_val',\
                                   'vendorid_val',\
                                   'improvement_surcharge_val',\
                                   'mta_tax_val',\
                                   'extra_val',\
                                   'fare_amount_val',\
                                   'tip_amount_val',\
                                   'tolls_amount_val',\
                                   'total_amount_val',\
                                   'store_and_fwd_flag_val',\
                                   'trip_distance_val',\
                                   'passenger_count_val',\
                                   'payment_type_val',\
                                   'ratecodeid_val',\
                                   'congestion_surcharge_val',\
                                   'row_val'])

        # We can drop duplicates in the clean dataset here - they will all have pickup and dropoff times or locations
        # The times are stored to the millisecond and PySpark's drop_duplicates compares all columns
        # If two records have pickup and dropoff times identical to the millisecond, we can safely drop them
        # If two records have both location ids identical and identical dispatch numbers, we can also safely drop them
        duplicates_result = drop_dupes(dfiClean)
        dfiClean = duplicates_result[0]

        dropped_duplicates_count = dropped_duplicates_count + duplicates_result[1]

        dfiClean_row_count = duplicates_result[2]
        clean_count = clean_count + dfiClean_row_count

        # Save the cleaned dataframes for repairing later
        save_df(dfiClean,ntpath.basename(i) + 'clean', clean_YELLOW_directory)
        
        dirtyInI = dfi.filter((dfi.row_val == 1) | (dfi.row_val == 2))\
            .withColumn("filename", lit(ntpath.basename(i)))
        
        # We then add the dirty records to a spark dataframe, with an additional column specifying file of origin
        if i == yellow_integratedPaths[0]:
            dfiDirty = dirtyInI

        else:
            dfiDirty = dfiDirty.unionByName(dirtyInI)

        dirtyInI = dirtyInI.withColumn("filename", col('filename').cast("string"))
        save_df(dirtyInI,ntpath.basename(i) + 'dirty', dirty_YELLOW_directory)
        
        print(str(dfiClean_row_count) + ' clean records preserved out of ' + str(dfi_row_count) + \
              ' original records - ' + str(dfiClean_row_count/dfi_row_count*100)[0:4] + "%")
        print(str(dirtyInI.filter(dirtyInI.row_val == 1).count()) + " invalid records have been found")
        print(str(dirtyInI.filter(dirtyInI.row_val == 2).count()) + " records with missing data have been found")
        
        # We can use Pandas to keep track of what's going on - this really slows down the code, but statistics are nice.
        # Note: if you comment this out to make the code run faster, do not forget to delete the return

        b = dfi.filter(col('tpep_dropoff_datetime_val')==2).count()
        c = dfi.filter(col('tpep_pickup_datetime_val')==2).count()
        d = dfi.filter(col('dolocationid_val')==2).count()
        e = dfi.filter(col('pulocationid_val')==2).count()
        f = dfi.filter(col('vendorid_val')==2).count()
        g = dfi.filter(col('ratecodeid_val')==2).count()
        h = dfi.filter(col('payment_type_val')==2).count()
        j = dfi.filter(col('store_and_fwd_flag_val')==2).count()
        k = dfi.filter(col('trip_distance_val')==2).count()
        l = dfi.filter(col('passenger_count_val')==2).count()
        m = dfi.filter(col('tip_amount_val')==2).count()
        n = dfi.filter(col('fare_amount_val')==2).count()
        o = dfi.filter(col('total_amount_val')==2).count()
        p = dfi.filter(col('improvement_surcharge_val')==2).count()
        q = dfi.filter(col('extra_val')==2).count()
        r = dfi.count()
        s = dfi.filter(col('mta_tax_val')==2).count()
        
        u = dfi.filter(col('tolls_amount_val')==2).count()
        w = dfi.filter(col('congestion_surcharge_val')==2).count()
        x = dfi.filter(col('row_val')==2).count()
        
        missingLst.append([i,r,b,c,d,e,f,g,h,j,k,l,m,n,o,p,q,s,u,w,x])
        
        b1 = dfi.filter(col('tpep_dropoff_datetime_val')==1).count()
        c1 = dfi.filter(col('tpep_pickup_datetime_val')==1).count()
        d1 = dfi.filter(col('dolocationid_val')==1).count()
        e1 = dfi.filter(col('pulocationid_val')==1).count()
        f1 = dfi.filter(col('vendorid_val')==1).count()
        g1 = dfi.filter(col('ratecodeid_val')==1).count()
        h1 = dfi.filter(col('payment_type_val')==1).count()
        j1 = dfi.filter(col('store_and_fwd_flag_val')==1).count()
        k1 = dfi.filter(col('trip_distance_val')==1).count()
        l1 = dfi.filter(col('passenger_count_val')==1).count()
        m1 = dfi.filter(col('tip_amount_val')==1).count()
        n1 = dfi.filter(col('fare_amount_val')==1).count()
        o1 = dfi.filter(col('total_amount_val')==1).count()
        p1 = dfi.filter(col('improvement_surcharge_val')==1).count()
        q1 = dfi.filter(col('extra_val')==1).count()
        r1 = dfi.count()
        s1 = dfi.filter(col('mta_tax_val')==1).count()
        
        u1 = dfi.filter(col('tolls_amount_val')==1).count()
        w1 = dfi.filter(col('congestion_surcharge_val')==1).count()
        x1 = dfi.filter(col('row_val')==1).count()
        
        dirtyLst.append([i,r1,b1,c1,d1,e1,f1,g1,h1,j1,k1,l1,m1,n1,o1,p1,q1,s1,u1,w1,x1])
    
    PdfMissing=pd.DataFrame(missingLst,columns=['file','number of lines','tpep_dropoff_datetime_val','tpep_pickup_datetime_val',\
                                 'dolocationid_val','pulocationid_val','vendorid_val','ratecodeid_val',\
                                 'payment_type_val','store_and_fwd_flag_val','trip_distance_val',\
                                 'passenger_count_val', 'tip_amount_val','fare_amount_val','total_amount_val',\
                                 'improvement_surcharge_val','extra_val','mta_tax_val','tolls_amount_val',\
                                 'congestion_surcharge_val','row_val'])
    
    #Only the columns with at least one value different from zero are kept.
    PdfMissing = PdfMissing.loc[:, (PdfMissing != 0).any(axis=0)] 

    PdfDirty=pd.DataFrame(dirtyLst,columns=['file','number of lines','tpep_dropoff_datetime_val','tpep_pickup_datetime_val',\
                             'dolocationid_val','pulocationid_val','vendorid_val','ratecodeid_val',\
                             'payment_type_val','store_and_fwd_flag_val','trip_distance_val',\
                             'passenger_count_val', 'tip_amount_val','fare_amount_val','total_amount_val',\
                             'improvement_surcharge_val','extra_val','mta_tax_val','tolls_amount_val',\
                             'congestion_surcharge_val','row_val'])

    #Only the columns with at least one value different from zero are kept.
    PdfDirty = PdfDirty.loc[:, (PdfDirty != 0).any(axis=0)] 
        
    print("Out of " + str(row_count) + " original records, " + str(clean_count) + " records were already clean enough to use in queries.")
    print(str(dropped_duplicates_count) + " records were dropped as duplicates.")
    print(str(row_count - clean_count - dropped_duplicates_count) \
          + " dirty records remain for resolution.")
    
    return dfiDirty, PdfMissing, PdfDirty

In [138]:
yellow_cleanup_return = yellow_cleanup()
dirtyYELLOW = yellow_cleanup_return[0]
missingYellow = yellow_cleanup_return[1]
dirtyDetailedYellow = yellow_cleanup_return[2]

opened /home/epb123/output/integrated Yellow/yellow_tripdata_2009-01.csv
0 duplicate entries removed
0 clean records preserved out of 28187 original records - 0.0%
28187 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2009-02.csv
0 duplicate entries removed
0 clean records preserved out of 26763 original records - 0.0%
26763 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2009-03.csv
0 duplicate entries removed
0 clean records preserved out of 28782 original records - 0.0%
28782 invalid records have been found
0 records with missing data have been found
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2009-04.csv
0 duplicate entries removed
0 clean records preserved out of 28599 original records - 0.0%
28599 invalid records have been found
0 records with missing data have been found
opened /home/epb

In [139]:
# Clean the PySpark outputs from before
YELLOW_cleanCSV_folderPath = cleanup(clean_YELLOW_directory)
with open(v_direc + "YELLOW_cleanCSV_folderPath",'wb') as cleanedYELLOWCSVdirec:
    pickle.dump(YELLOW_cleanCSV_folderPath,cleanedYELLOWCSVdirec)

YELLOW_dirtyCSV_folderPath = cleanup(dirty_YELLOW_directory)
with open(v_direc + "YELLOW_dirtyCSV_folderPath",'wb') as dirtyYELLOWCSVdirec:
    pickle.dump(YELLOW_dirtyCSV_folderPath,dirtyYELLOWCSVdirec)

folder already exists at /home/epb123/output/cleaned YELLOW/all CSV
folder already exists at /home/epb123/output/dirty YELLOW/all CSV


Based on filtering for records with all of the information needed for the queries, we appear to here have 2,131,041 records for cleaning - a number which might shock. Many of this could have come from the payment_type filter - a lot of early yellow records did not use correct payment types.

Let us examine the data for now and see what can be repaired.

In [140]:
dirtyDetailedYellow

,file,number of lines,tpep_dropoff_datetime_val,tpep_pickup_datetime_val,dolocationid_val,pulocationid_val,vendorid_val,ratecodeid_val,payment_type_val,store_and_fwd_flag_val,...,passenger_count_val,tip_amount_val,fare_amount_val,total_amount_val,improvement_surcharge_val,extra_val,mta_tax_val,tolls_amount_val,congestion_surcharge_val,row_val
0,/home/epb123/output/integrated Yellow/yellow_t...,28187,575,575,585,563,28187,0,28187,2,...,4,0,0,0,0,0,0,0,0,28187
1,/home/epb123/output/integrated Yellow/yellow_t...,26763,201,201,571,554,26763,0,26763,8584,...,0,0,0,1,0,0,0,0,0,26763
2,/home/epb123/output/integrated Yellow/yellow_t...,28782,111,111,613,578,28782,0,28782,10322,...,0,0,0,1,0,0,0,0,0,28782
3,/home/epb123/output/integrated Yellow/yellow_t...,28599,120,120,622,613,28599,0,28599,12891,...,0,0,0,2,0,0,0,0,0,28599
4,/home/epb123/output/integrated Yellow/yellow_t...,29606,120,120,637,657,29606,0,29606,13316,...,0,0,0,3,0,0,0,0,0,29606
5,/home/epb123/output/integrated Yellow/yellow_t...,28375,97,97,462,460,28375,0,28375,12809,...,0,0,0,0,0,0,0,0,0,28375
6,/home/epb123/output/integrated Yellow/yellow_t...,27260,78,78,462,451,27260,0,27260,12218,...,1,0,0,3,0,0,0,0,0,27260
7,/home/epb123/output/integrated Yellow/yellow_t...,27377,93,93,482,482,27377,0,27377,12385,...,0,0,0,4,0,0,0,0,0,27377
8,/home/epb123/output/integrated Yellow/yellow_t...,27974,99,99,482,494,27974,0,27974,12409,...,0,0,0,1,0,2,0,0,0,27974
9,/home/epb123/output/integrated Yellow/yellow_t...,31216,96,96,612,608,31216,0,31216,13989,...,0,0,0,6,0,0,0,0,0,31216


In [141]:
dirtyDetailedYellow.columns.tolist()

['file',
 'number of lines',
 'tpep_dropoff_datetime_val',
 'tpep_pickup_datetime_val',
 'dolocationid_val',
 'pulocationid_val',
 'vendorid_val',
 'ratecodeid_val',
 'payment_type_val',
 'store_and_fwd_flag_val',
 'trip_distance_val',
 'passenger_count_val',
 'tip_amount_val',
 'fare_amount_val',
 'total_amount_val',
 'improvement_surcharge_val',
 'extra_val',
 'mta_tax_val',
 'tolls_amount_val',
 'congestion_surcharge_val',
 'row_val']

In [142]:
missingYellow

,file,number of lines,dolocationid_val,vendorid_val,ratecodeid_val,payment_type_val,store_and_fwd_flag_val,passenger_count_val,mta_tax_val,congestion_surcharge_val,row_val
0,/home/epb123/output/integrated Yellow/yellow_t...,28187,0,0,28187,0,28185,0,28187,28187,0
1,/home/epb123/output/integrated Yellow/yellow_t...,26763,0,0,26763,0,18179,0,26763,26763,0
2,/home/epb123/output/integrated Yellow/yellow_t...,28782,0,0,28782,0,18460,0,28782,28782,0
3,/home/epb123/output/integrated Yellow/yellow_t...,28599,0,0,28599,0,15708,0,28599,28599,0
4,/home/epb123/output/integrated Yellow/yellow_t...,29606,0,0,29606,0,16290,0,29606,29606,0
5,/home/epb123/output/integrated Yellow/yellow_t...,28375,0,0,28375,0,15566,0,28375,28375,0
6,/home/epb123/output/integrated Yellow/yellow_t...,27260,0,0,27260,0,15042,0,27260,27260,0
7,/home/epb123/output/integrated Yellow/yellow_t...,27377,0,0,27377,0,14992,0,27377,27377,0
8,/home/epb123/output/integrated Yellow/yellow_t...,27974,0,0,27974,0,15565,0,27974,27974,0
9,/home/epb123/output/integrated Yellow/yellow_t...,31216,0,0,31216,0,17227,0,25280,31216,0


In [143]:
missingYellow.columns.tolist()

['file',
 'number of lines',
 'dolocationid_val',
 'vendorid_val',
 'ratecodeid_val',
 'payment_type_val',
 'store_and_fwd_flag_val',
 'passenger_count_val',
 'mta_tax_val',
 'congestion_surcharge_val',
 'row_val']

It appears like a good number of those total_amount errors are arising from beyond 2019 - since the "extra" column also shows a significant uptick there, I suspect the two are related.  

The results of a brief look directly in the records seems to support this - it looks like at some point after 2019, congestion_surcharge might be getting picked up by extra (as well as some other oddities in extra) as well as by itself, throwing off our filter. We will investigate further soon, but the fact that congestion_surcharges appear to a) start being incorrect and b) stop being missing around the same time adds weight to this hypothesis. 

We also here appear to have a number of missing dolocationid_val clustered in one file. Let's investigate later.

In [144]:
# Store the dirty dataframe separately to avoid unfortunate errors
# We can update our cleaned values as we go

recleanedYELLOW = dirtyYELLOW

Out of our core variables, what is broken?

In [145]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.row_val == 1).count())+" rows containing dirty values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.row_val == 2).count())+" rows containing no dirty values but missing values")

We have 2130560 rows containing dirty values and...
481 rows containing no dirty values but missing values


Looking at the summary, we can see that out of what is important, we still have quite a good set:  
- pickup_datetime: clean, not missing
- dropoff_datetime: clean, not missing
- pulocationid: dirty, not missing
- dolocationid: dirty, missing
- trip_distance: dirty, not missing
- tip_amount: dirty, not missing
- total_amount: dirty, not missing
- payment_type: dirty, missing

Let us first dive deeper into those missing values of dolocationid...  

In [146]:
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.dolocationid_val == 2).count())+" missing dropoff values")

We have 25 missing dropoff values


Since there were so few records, we may as well pull them out and take a look.

In [147]:
missingYellow[['dolocationid_val']].loc[(missingYellow[['dolocationid_val']].\
                                         loc[:, missingYellow[['dolocationid_val']].dtypes != object] != 0).any(1)]

,dolocationid_val
14,11
19,1
25,1
40,1
41,1
47,4
48,1
54,2
57,1
58,2


In [148]:
# The code is slightly ugly, but it functions for our purposes
doMiss = create_df(missingYellow.at[14,'file'])
doMiss = doMiss.unionByName(create_df(missingYellow.at[19,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[25,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[40,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[41,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[47,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[48,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[54,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[57,'file']))
doMiss = doMiss.unionByName(create_df(missingYellow.at[58,'file']))

doMiss.filter(col('dolocationid').isNull()).select(['dolocationid','pulocationid','total_amount','tip_amount',\
                                                    'tpep_pickup_datetime','tpep_dropoff_datetime']).show()

opened /home/epb123/output/integrated Yellow/yellow_tripdata_2010-03.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2010-08.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2011-02.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2012-05.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2012-06.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2012-12.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2013-01.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2013-07.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2013-10.csv
opened /home/epb123/output/integrated Yellow/yellow_tripdata_2013-11.csv
+------------+---------------+------------------+----------+--------------------+---------------------+
|dolocationid|   pulocationid|      total_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+------------+---------------+------------------+----------+--

It appears as though a large part of the records with a missing dolocationid also have a 0 total_amount and a 0.5 tip_amount. I would suspect data corruption or some repeated incorrect entries as the cause of this error - we should remove the records as their inclusion would likely be inaccurate.  

For the other records, it looks as if the dolocationid was truly null - based on the 2.2 code, this would mean our query point for the shapefile was invalid. Perhaps the dropoff was underground in such a way that the taxi's GPS could not store the record correctly.

In [149]:
print(str(recleanedYELLOW.filter((col('dolocationid').isNull()) \
                                         & (col('total_amount') == 0) & (col('tip_amount') == 0.5)).count()) \
         + " records are dropped as a result of this action.")
recleanedYELLOW = recleanedYELLOW.filter((col('dolocationid').isNotNull()) \
                                         | (col('total_amount') != 0) | (col('tip_amount') != 0.5))

# We also want to update the dirty database for later use
dirtyYELLOW = dirtyYELLOW.filter((col('dolocationid').isNotNull()) \
                                         | (col('total_amount') != 0) | (col('tip_amount') != 0.5))

11 records are dropped as a result of this action.


We can start again on the dirty records with **total_amount**

In [150]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.total_amount_val == 1).count())+" dirty values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.total_amount_val == 2).count())+" missing values")

We have 36354 dirty values and...
0 missing values


In [151]:
# We can then analyse the dirty values - we know the 5% criterion will have caused many records to drop
# Let us now check to see if the values are incorrect strings
# We know the total_amounts here are not null, thus we can do...

dirtyYELLOWanalysis = dirtyYELLOW.filter(dirtyYELLOW.total_amount_val == 1)
print("we have " + str(dirtyYELLOWanalysis.select("total_amount",sql.col("total_amount").cast("float").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [152]:
# We can then continue with negatives

dirtyYELLOWanalysis = dirtyYELLOWanalysis.withColumn('total_amount',col('total_amount').cast(FloatType()))
dirtyYELLOWanalysis = dirtyYELLOWanalysis.withColumn('negative_total', sql.when(col('total_amount') < 0.0, 1).otherwise(0))
print("we appear to have " + str(dirtyYELLOWanalysis.filter(col('negative_total') == 1).count()) + " negative total_amount entries")

we appear to have 693 negative total_amount entries


Negative values can be converted into positive ones, under the assumption that they are mistakenly entered. If we then remove duplicates after the conversion, we can cover ourselves in case the negative values are some sort of accounting reversal.  

In [153]:
# Let us now check to see if any of the records tripped the 1000 upper bound

print("there are "+str(dirtyYELLOWanalysis.filter(dirtyYELLOWanalysis.total_amount > 1000).count())+" lucky taxi drivers")
dirtyYELLOWanalysis.filter(dirtyYELLOWanalysis.total_amount > 1000).select([col('filename'),col('total_amount')]).show()

there are 3 lucky taxi drivers
+--------------------+------------+
|            filename|total_amount|
+--------------------+------------+
|yellow_tripdata_2...|     9001.01|
|yellow_tripdata_2...|    200006.3|
|yellow_tripdata_2...|      1239.3|
+--------------------+------------+



For the lucky taxi drivers, the record with a 1,200 fare could be a visit by the same YouTuber as we found in Green, but the other two are highly unlikely. We will thus set those values to Null.

All other conditions were checked, so we must have approx. 36k total_amount records which are more than 5% away from the sum of their components. We will follow the same philosophy as last time, but bearing in mind the possible error in the extra column, and the quantity of broken records, we should run the validation once more, but with a further condition checking the relationship with the congestion_surcharge removed - if, as I suspect, it was occasionally being double counted, this would greatly reduce the number of flagged values.

In [154]:
# Let's make the updates to the recleanedYELLOW, which we will later inject back into the dataset.

# We have no strings to update - we can cast the column as a float
recleanedYELLOW = recleanedYELLOW.withColumn('total_amount',col('total_amount').cast(FloatType()))
# So first update values as absolute - probably faster to abs() the whole column than rely on conditionals
recleanedYELLOW = recleanedYELLOW.withColumn('total_amount', sql.abs(col('total_amount')))
# We reject the discovered outliers with values 200,006.3 and 9001.01 - let us remove them
recleanedYELLOW = recleanedYELLOW.withColumn('total_amount', sql.when(((col('total_amount') == 200006.3)\
                                            | (col('total_amount') == 9001.01)),\
                                            None).otherwise(col('total_amount')))
# Let us set the remainder of records = Null, after running our additional test

def YELLOWtotalupdate(total,cong_s,extra,fare,improv,tax,tip,toll):
# Function to transform total_amount to null if conditions not held
# Function returns Null if it is invalid, total if it is valid
    
    #no need for this as we know it's not missing
#     if total == None or total == "" or total == 'nan':
#         return 2
#     try:
#         total = float(total)
#     except:
#         return 1 
    # No need for this as already done
    #difference = abs(total)
    difference = total
    for i in [cong_s,extra,fare,improv,tax,tip,toll]:
        if i == None or i == "" or i == 'nan' or i == 'NaN':
            i = 0
        try:
            i = abs(float(i))
        # Note we use abs here so the fact that we haven't checked tip yet does not affect it
            difference = difference - i
        except:
        # We do the same for float values - we simply want to ignore incorrect strings here
            pass
        
    if abs(difference) > total*0.05: #and total >= 0 and total <= 1000:
    # if the difference is greater than 5% of total, it is incorrect
    # before returning None, test to see that the congestion_surcharge has no impact    
        
        difference = total
        for i in [extra,fare,improv,tax,tip,toll]:
            if i == None or i == "" or i == 'nan' or i == 'NaN':
                i = 0
            try:
                i = abs(float(i))
                difference = difference - i
            except:
                pass  
        
        if abs(difference) > total*0.05: #and total >= 0 and total <= 1000:
            return None
    
    return total

# Define function as a UDF for pyspark usage
udfYELLOWtotalbroken = udf(YELLOWtotalupdate, FloatType())

print("Receipt updating testing function stored as UDF")

def add_YELLOW_total_amount_update(dfrval):
# Adds the extra validation as new dataframe columns, calculated from other columns in the dataframe
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfrval = dfrval.withColumn('total_amount', udfYELLOWtotalbroken('total_amount','congestion_surcharge','extra',\
    'fare_amount','improvement_surcharge','mta_tax','tip_amount','tolls_amount'))

    return dfrval

recleanedYELLOW = add_YELLOW_total_amount_update(recleanedYELLOW)

# we only need to count once - we did not previously have missing values
print(str(recleanedYELLOW.filter(col('total_amount').isNull()).count()) + " irreconcilable totals removed")

Receipt updating testing function stored as UDF
3560 irreconcilable totals removed


Out of 36,365 flagged total_amount values, we only end up removing 3,560 of them. Assuming congestion_surcharge was double counted has made a huge impact.

We can then move onto **tip_amount**

In [155]:
# Let's find out where any dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.tip_amount_val == 1).count())+" dirty values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.tip_amount_val == 2).count())+" missing values")

We have 8 dirty values and...
0 missing values


In [156]:
# Let us now check to see if the values are incorrect strings
# We know the tip_amounts here are not null, thus we can do...

dirtyYELLOWanalysis = dirtyYELLOW.filter(dirtyYELLOW.tip_amount_val == 1)
print("we have " + str(dirtyYELLOWanalysis.select("tip_amount",sql.col("tip_amount").cast("float").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [157]:
# We can then continue with negatives

dirtyYELLOWanalysis = dirtyYELLOWanalysis.withColumn('tip_amount',col('tip_amount').cast(FloatType()))
dirtyYELLOWanalysis = dirtyYELLOWanalysis.withColumn('negative_tip', sql.when(col('tip_amount') < 0.0, 1).otherwise(0))
print("we appear to have " + str(dirtyYELLOWanalysis.filter(col('negative_tip') == 1).count()) + " negative tip_amount entries")

we appear to have 8 negative tip_amount entries


As we had 3 in Green, we have 8 in Yellow - we can repair these and put them back into the dataset.

In [158]:
# Let's make the updates to the recleanedYELLOW, which we will later inject back into the dataset.

# We have no strings to update - we can cast the column as a float
recleanedYELLOW = recleanedYELLOW.withColumn('tip_amount',col('tip_amount').cast(FloatType()))
# So first update values as absolute - probably faster to abs() the whole column than rely on conditionals
recleanedYELLOW = recleanedYELLOW.withColumn('tip_amount', sql.abs(col('tip_amount')))

Let's start on **payment_type**

In [159]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.payment_type_val == 1).count())+" dirty values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.payment_type_val == 2).count())+" missing values")

We have 2072517 dirty values and...
505 missing values


We have found the cause of our number of dirty records... We assume the payment_type started being stored in number format only after a certain date. Let's, however, start with missing values.

We can impute missing values - we have just fixed the tip amount  
- If there is a Null value but tip exists, we store as card  
- If there is a Null value but tip == 0, we store as cash  

We assume the tip would not exist with a void trip or a no charge trip. We also make the assumption that no tip strongly correlates with a cash trip (in a country where restaurant tips are 20% standard, we believe this is reasonable)

In [160]:
# We can impute missing values - we have just fixed the tip amount
# If there is a Null value but tip exists, we store as card
# If there is a Null value but tip == 0, we store as cash
# We assume the tip would not exist with a void trip or a no charge trip
dirtyYELLOWanalysis = dirtyYELLOW
print("we have " + str(dirtyYELLOWanalysis.filter((col("payment_type").isNull()) & (col("tip_amount") != 0)).count())\
                       + " trips with a tip but no payment type stored")
print("we have " + str(dirtyYELLOWanalysis.filter((col("payment_type").isNull()) & (col("tip_amount") == 0)).count())\
                       + " trips with no tip and no payment type stored")

we have 115 trips with a tip but no payment type stored
we have 390 trips with no tip and no payment type stored


In [161]:
recleanedYELLOW = recleanedYELLOW.withColumn('payment_type',sql.when((col("payment_type").isNull())\
                                                                   & (col("tip_amount") != 0), '1')\
                                           .otherwise(col('payment_type')))
recleanedYELLOW = recleanedYELLOW.withColumn('payment_type',sql.when((col("payment_type").isNull())\
                                                                   & (col("tip_amount") == 0), '2')\
                                           .otherwise(col('payment_type')))

In [162]:
print("we have " + str(recleanedYELLOW.filter((col("payment_type").isNull()) & (col("tip_amount") != 0)).count())\
                       + " remaining trips with a tip but no payment type stored")
print("we have " + str(recleanedYELLOW.filter((col("payment_type").isNull()) & (col("tip_amount") == 0)).count())\
                       + " remaining trips with no tip and no payment type stored")

we have 0 remaining trips with a tip but no payment type stored
we have 0 remaining trips with no tip and no payment type stored


In [163]:
dirtyYELLOWanalysis = dirtyYELLOW.filter(dirtyYELLOW.payment_type_val == 1)

# We can test invalid payment types
# Let us first check to see if the values are incorrect strings
# We know the payment_types here are not null, thus we can do...

print("we have " + str(dirtyYELLOWanalysis.filter(col("payment_type").cast("int").isNull()).count())\
                       + " incorrectly stored strings")

we have 2067593 incorrectly stored strings


In [164]:
def paymentStringFix(pmt):
# Function returns 0 if the figure is valid, 1 if it is invalid, 2 if it is None
# We have no more null values    

    pmt = pmt.lower()
    
# 1) Credit card
    if pmt[0:2] == 'cr' or pmt[0:3] == 'car':
        return '1'
# 2) Cash 
    if pmt[0:2] == 'cs' or pmt[0:3] == 'cas':
        return '2'
# 3) No charge
    if pmt[0] == 'n':
        return '3'
# 4) Dispute
    if pmt[0] == 'd':
        return '4'
# 5) Unknown
    if pmt[0] == 'u':
        return '5'
# 6) Voided trip
    if pmt[0] == 'v':
        return '6'

    # if it is none of the above, just return the original string
    return pmt
    
# Define function as a UDF for pyspark usage
udfpayment_string_fix = udf(paymentStringFix, StringType())

print("payment testing function stored as UDF")

def add_payment_string_fix(dfTXval):
# Takes a dataframe
# Returns that dataframe with modifications
    
    dfTXval = dfTXval.withColumn('payment_type', udfpayment_string_fix('payment_type'))

    return dfTXval

tax testing function stored as UDF


In [165]:
recleanedYELLOW = add_payment_string_fix(recleanedYELLOW)

In [166]:
print("we have " + str(recleanedYELLOW.filter(col("payment_type").cast("int").isNull()).count())\
                       + " incorrectly stored strings remaining")

we have 0 incorrectly stored strings remaining


Now that the strings are dealt with, we can test valid entries

In [167]:
print("We have " + str(dirtyYELLOWanalysis.filter(col("payment_type").cast("int") > 6).count())\
                       + " values above 6")
print("We have " + str(dirtyYELLOWanalysis.filter(col("payment_type").cast("int") < 1).count())\
                       + " values below 1")

We have 0 values above 6
We have 0 values below 1


We should check how many trips were either 3 or 6:  
1) Credit card  
2) Cash  
3) No charge  
4) Dispute  
5) Unknown  
6) Voided trip  

Now that we have mostly repaired payment type, We have flagged trips in 3 or 6 for removal because they are invalid - we do not want to analyse free trips or trips which did not exist, particularly with queries based on amounts. Trips in unknown, we cannot classify, and just because someone decided to dispute their fare did not mean by itself that it was not valid.

In [168]:
print("We will need to drop "+str(dirtyYELLOWanalysis.filter(col('payment_type') == 3).count())\
      + " trips with no charge.")
print("We will need to drop "+str(dirtyYELLOWanalysis.filter(col('payment_type') == 6).count())\
      + " voided trips.")

We will need to drop 4917 trips with no charge.
We will need to drop 0 voided trips.


In [169]:
recleanedYELLOW = recleanedYELLOW.filter(col('payment_type') != 3)
recleanedYELLOW = recleanedYELLOW.filter(col('payment_type') != 6)

In [170]:
print("We now have "+str(recleanedYELLOW.filter(col('payment_type') == 3).count())\
      + " trips with no charge.")
print("We now have "+str(recleanedYELLOW.filter(col('payment_type') == 6).count())\
      + " voided trips.")

We now have 0 trips with no charge.
We now have 0 voided trips.


In [171]:
print("We also need to clean " + str(recleanedYELLOW.filter((col("payment_type")== 2) & (col("tip_amount") != 0)).count())\
                       + " cash trips with a recorded tip")

We also need to clean 444 cash trips with a recorded tip


Here, we should change the payment_type to 1:

In [172]:
recleanedYELLOW = recleanedYELLOW.withColumn('payment_type',sql.when(\
                                                                     (col("payment_type")== 2) &\
                                                                     (col("tip_amount") != 0),\
                                                                     1).otherwise(col('payment_type')))

In [173]:
print("We now have " + str(recleanedYELLOW.filter((col("payment_type")== 2) & (col("tip_amount") != 0)).count())\
                       + " cash trips with a recorded tip")

We now have 0 cash trips with a recorded tip


In [174]:
# We have been dropping rows in our cleaned file, so let's refresh dirtyYELLOW
dirtyYELLOW = recleanedYELLOW

We can then move onto **trip_distance**

In [175]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.trip_distance_val == 1).count())+" dirty values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.trip_distance_val == 2).count())+" missing values")

We have 19367 dirty values and...
0 missing values


In [176]:
# Let us now check to see if the values are incorrect strings
# We know the trip_distances here are not null, thus we can do...

dirtyYELLOWanalysis = dirtyYELLOW.filter(dirtyYELLOW.trip_distance_val == 1)
print("we have " + str(dirtyYELLOWanalysis.select("trip_distance",sql.col("trip_distance").cast("float").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 0 incorrectly stored strings


In [177]:
# We can then continue with negatives

dirtyYELLOWanalysis = dirtyYELLOWanalysis.withColumn('trip_distance',col('trip_distance').cast(FloatType()))
dirtyYELLOWanalysis = dirtyYELLOWanalysis.withColumn('negative_trip', sql.when(col('trip_distance') < 0.0, 1).otherwise(0))
print("we appear to have " + str(dirtyYELLOWanalysis.filter(col('negative_trip') == 1).count()) + " negative trip_distance entries")

we appear to have 3 negative trip_distance entries


In [178]:
print("there are "+str(dirtyYELLOWanalysis.filter(dirtyYELLOWanalysis.trip_distance == 0).count())+" trips to nowhere")
#dirtyYELLOWanalysis.filter(dirtyYELLOWanalysis.trip_distance == 0).select([col('filename'),col('trip_distance')]).show()

there are 19360 trips to nowhere


In [179]:
# Last is to check trips with distance >= 300
print("there are "+str(dirtyYELLOWanalysis.filter(sql.abs(dirtyYELLOWanalysis.trip_distance) >= 300.0).count())+" trips to somewhere far far away")
dirtyYELLOWanalysis.filter(sql.abs(dirtyYELLOWanalysis.trip_distance) >= 300).select([col('filename'),col('trip_distance'),col('total_amount')]).show()

there are 4 trips to somewhere far far away
+--------------------+-------------+------------+
|            filename|trip_distance|total_amount|
+--------------------+-------------+------------+
|yellow_tripdata_2...|        494.4|       74.21|
|yellow_tripdata_2...|        500.0|        23.8|
|yellow_tripdata_2...|     300833.1|        70.0|
|yellow_tripdata_2...|        396.3|      411.06|
+--------------------+-------------+------------+



Three of those could possibly be feasable, and their presence could in fact match with some of the total_amount anomalies we discovered, the record with a distance of >300k miles suggests a driver who never turned off his meter rather than a correct record. Out of the others, the 396.3 miles record could have been a driver going from New York to Buffalo - a trip of approximately 370 miles, and if the total amount matches the 1,239.3 value we saw earlier, I will keep it.  

"Plus 50 cents per 1/5 mile when traveling above 12mph or per 60 seconds in slow traffic or when the vehicle is stopped."
Is what the current NYC fare rate is - a total amount of only 400 does not align with this.

We now follow a similar pattern as green - we will correct negative values and then use tips and totals to filter the distance values (a trip with distance 0 but a tip certainly did not have the correct distance value), following with a check of the long trip.

In [180]:
# Let's make the updates to the recleanedYELLOW, which we will later inject back into the dataset.

# let's first handle that negative value
# Abs-ing the entire column is a little bit inelegant, but will suffice

# We have no strings to update - we can cast the column as a float
recleanedYELLOW = recleanedYELLOW.withColumn('trip_distance',col('trip_distance').cast(FloatType()))
# So first update values as absolute - probably faster to abs() the whole column than rely on conditionals
recleanedYELLOW = recleanedYELLOW.withColumn('trip_distance', sql.abs(col('trip_distance')))

# Count the dirty records - use the pre-filtered dataset here
print("we have "+str(recleanedYELLOW.filter(( col('trip_distance') == 0.0)\
            & ( (col('total_amount') >= 5.0) | (col("tip_amount") > 0.0))).count())+\
            " trips with 0 distances and a nonzero tip or an amount > $5")

# Next let's apply our filtering criteria to those trip_distances
recleanedYELLOW = recleanedYELLOW.withColumn('trip_distance', sql.when( ( col('trip_distance') == 0.0)\
                                            & ( (col('total_amount') >= 5.0) | (col("tip_amount") > 0.0)),\
                                            None).otherwise(col('trip_distance')))

print("we have "+str(recleanedYELLOW.filter(( col('trip_distance') == 0.0)\
            & ( (col('total_amount') >= 5.0) | (col("tip_amount") > 0.0))).count())+\
            " expensive trips with 0 distances and a nonzero tip or an amount > $5 remaining")

we have 12419 trips with 0 distances and a nonzero tip or an amount > $5
we have 0 expensive trips with 0 distances and a nonzero tip or an amount > $5 remaining


In [181]:
# This is a little bit disappointing, but we must remove these large distance values
recleanedYELLOW = recleanedYELLOW.withColumn('trip_distance', sql.when((col('trip_distance') == 396.3) \
                                            | ( col('trip_distance') == 494.4) \
                                            | ( col('trip_distance') == 500) \
                                            | ( col('trip_distance') == 300833.1),\
                                            None).otherwise(col('trip_distance')))

Finally, we can analyse the **dolocationid** and **pulocationid**

In [182]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.dolocationid_val == 1).count())+" dirty dropoff values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.dolocationid_val == 2).count())+" missing dropoff values, of which 11 have been removed.")
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.pulocationid_val == 1).count())+" dirty pickup values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.pulocationid_val == 2).count())+" missing pickup values")

We have 69851 dirty dropoff values and...
14 missing dropoff values, of which 11 have been removed.
We have 66229 dirty pickup values and...
0 missing pickup values


Again - we have run a filter in 2.2 - Null dropoff values likely were invalid points in the conversion.

Let's then start with **pulocationid**

In [183]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...

dirtyYELLOWanalysis = dirtyYELLOW.filter(col('pulocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyYELLOWanalysis.select("pulocationid",sql.col("pulocationid").cast("int").isNotNull().alias("strings")).\
        filter(col('strings') == 'false').count()) + " incorrectly stored strings")

we have 56548 incorrectly stored strings


These would result from a "failed to match" failure coming out of the 2.2 locationid calculations. Basically, if the coordinates were corrupt - i.e. something was stored at 5 deg. longitude and 36 deg latitude, the shapefile would not identify the location as being in New York. We can set these equal to Null to avoid any later confusion.

In [184]:
# Testing the theory...
print("we have "+str(recleanedYELLOW.filter(col('pulocationid') == "failed to match").count())+" match failures")

# Next let's apply our filtering criteria to those locationids
recleanedYELLOW = recleanedYELLOW.withColumn('pulocationid', sql.when(col('pulocationid') == "failed to match",\
                                            None).otherwise(col('pulocationid')))

print("we have re-valued "+str(recleanedYELLOW.filter(col('pulocationid').isNull()).count())+" match failures")

we have 56548 match failures
we have re-valued 56548 match failures


We will have run the locationid testing code in 2.2, and the resulting errors must therefore come from values 264, 265, and above.

In [185]:
print("We have "+str(recleanedYELLOW.filter(col('pulocationid') > 263).count())+" values above our threshold")

We have 9681 values above our threshold


In [186]:
recleanedYELLOW = recleanedYELLOW.withColumn('pulocationid', sql.when(col('pulocationid') > 263,\
                                            None).otherwise(col('pulocationid')))
print(str(recleanedYELLOW.filter(col('pulocationid') > 263).count())+" values remain after cleaning")

0 values remain after cleaning


Now for **dolocationid**

In [187]:
# Let us now check to see if the values are incorrect strings
# We know the locationids here are not null, thus we can do...

dirtyYELLOWanalysis = dirtyYELLOW.filter(col('dolocationid_val') == 1)

#CASTING AS INT HERE!!!!!!
print("we have " + str(dirtyYELLOWanalysis.filter(col("dolocationid").cast("int").isNull()).count()) + " incorrectly stored strings")

we have 59548 incorrectly stored strings


In [188]:
# Testing the theory...
print("we have "+str(recleanedYELLOW.filter(col('dolocationid') == "failed to match").count())+" match failures")

# Next let's apply our filtering criteria to those locationids
recleanedYELLOW = recleanedYELLOW.withColumn('dolocationid', sql.when(col('dolocationid') == "failed to match",\
                                            None).otherwise(col('dolocationid')))

we have 59548 match failures


In [189]:
print("we have re-valued "+str(recleanedYELLOW.filter(col('dolocationid').isNull()).count()-14)+" match failures")

we have re-valued 59548 match failures


In [190]:
print("We have "+str(recleanedYELLOW.filter(col('dolocationid') > 263).count())+" values above our threshold")

We have 10303 values above our threshold


In [191]:
recleanedYELLOW = recleanedYELLOW.withColumn('dolocationid', sql.when(col('dolocationid') > 263,\
                                            None).otherwise(col('dolocationid')))

print(str(recleanedYELLOW.filter(col('dolocationid') > 263).count())+" values remain above the threshold")

0 values remain above the threshold


Let's not forget to examine the incorrect **tpep_pickup/dropoff_datetime** columns

In [192]:
# Let's find out where our dropped records came from
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.tpep_dropoff_datetime_val == 1).count())+" dirty dropoff values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.tpep_dropoff_datetime_val == 2).count())+" missing dropoff values")
print("We have "+str(dirtyYELLOW.filter(dirtyYELLOW.tpep_pickup_datetime_val == 1).count())+" dirty pickup values and...")
print(str(dirtyYELLOW.filter(dirtyYELLOW.tpep_pickup_datetime_val == 2).count())+" missing pickup values")

We have 8757 dirty dropoff values and...
0 missing dropoff values
We have 8757 dirty pickup values and...
0 missing pickup values


Given the prior issues with trip_distance, we can be reasonably confident that there will be further mismatches. Any trip with an average speed >75mph in central NYC will be an incorrect record - while the several long journeys we have found would allow us to assume someone might average 55 (speed limit in NY) or so on the highway, an average above 75 when so much time would be lost moving through NYC would be almost impossible. We will therefore first correct for any trips demonstrating this error - they will almost certainly be incorrect, and so we will drop the records. 

Along with this, we can correct for invalid formats (by setting to Null), and afterwards we can examine dropoffs before pickups (where we will repair by switching pickuptime and dropoff time).

In [193]:
del dirtyYELLOW
del dirtyYELLOWanalysis

# we no longer use these, and we need the memory. If I were re-writing this code, I would avoid their usage.

In [194]:
recleanedYELLOW = recleanedYELLOW.withColumn('dropdt', udfCOLORdatetime_drop_flag('tpep_pickup_datetime',\
                                                                                'tpep_dropoff_datetime',\
                                                                                'trip_distance'))
# Drop columns when datetime flag is True!
recleanedYELLOW = recleanedYELLOW.withColumn('tpep_dropoff_datetime',sql.when(col('dropdt') == True, None).\
                                           otherwise(col('tpep_dropoff_datetime')))
recleanedYELLOW = recleanedYELLOW.withColumn('tpep_pickup_datetime',sql.when(col('dropdt') == True, None).\
                                           otherwise(col('tpep_pickup_datetime')))

recleanedYELLOW = recleanedYELLOW.drop('dropdt')

In [195]:
# We can do this because we have no pre-existing Nulls

print("In the current version of the cleaned file, we now have... "+\
     str(recleanedYELLOW.filter(recleanedYELLOW.tpep_dropoff_datetime.isNull()).count())+\
     " fewer datetime values and "+\
     str(recleanedYELLOW.filter(recleanedYELLOW.tpep_pickup_datetime.isNull()).count())+\
     " fewer pickup values.")

In the current version of the cleaned file, we now have... 8254 fewer datetime values and 8254 fewer pickup values.


In [196]:
# Make the switch Flag dummy
recleanedYELLOW = recleanedYELLOW.withColumn('tsFlag',udfCOLORdatetime_switch('tpep_pickup_datetime','tpep_dropoff_datetime'))
print("We will now repair "+str(recleanedYELLOW.filter(col('tsFlag') == 1).count())+" backwards values")

# Put dropoff values into a temporary column
recleanedYELLOW = recleanedYELLOW.withColumn('dropoff_temp',col('tpep_dropoff_datetime'))

# Move pickup values into dropoff when switch flag is 1
recleanedYELLOW = recleanedYELLOW.withColumn('tpep_dropoff_datetime',sql.when(col('tsFlag') == 1,col('tpep_pickup_datetime')).\
                                           otherwise(col('tpep_dropoff_datetime')))
# Move dropoff values from temporary column into pickup when switch flag is 1
recleanedYELLOW = recleanedYELLOW.withColumn('tpep_pickup_datetime',sql.when(col('tsFlag') == 1,col('dropoff_temp')).\
                                           otherwise(col('tpep_pickup_datetime')))

# Drop temporary columns
recleanedYELLOW = recleanedYELLOW.drop(*['tsFlag','dropoff_temp'])

We will now repair 505 backwards values


We appear to have cleaned two more records than expected - as our criteria here are firm, we believe this could be the result of rounding or the prior repair to the trip distance.

While we still have further errors in other variables, our main set and the set relevant for the query has been fully analysed. We have replaced incorrect and irreparable values with Null, and repaired those which we could. We can now re-merge our recleanedYELLOW dataset with the other data.  

Let's start by checking how many complete (for 2.4) records we can extract:

In [197]:
# We've set irreparable values as Null, so...
print("We have managed to repair "+str(recleanedYELLOW.filter(\
                                                             (col('tpep_pickup_datetime').isNotNull()) &\
                                                             (col('tpep_dropoff_datetime').isNotNull()) &\
                                                             (col('dolocationid').isNotNull()) &\
                                                             (col('pulocationid').isNotNull()) &\
                                                             (col('trip_distance').isNotNull()) &\
                                                             (col('total_amount').isNotNull()) &\
                                                             (col('tip_amount').isNotNull()) &\
                                                             (col('payment_type').isNotNull())
                                                             ).count())+" complete records")

We have managed to repair 2030231 complete records


Given that this is approximately 31% of all of the dirty records we discovered, to have 40k complete and usable additions is a strong result. The question is then what we do with the remaining incomplete records. Given that this represents ~2% of the raw dataset, it would be nice to include them, but again we would have a constantly changing overall database size across queries. Our results would therefore be inconsistent.  

We thus proceed by returning these fully cleaned values to their respective datasets.

In [198]:
# Let us filter for the values we want
everdirtyYELLOW = recleanedYELLOW.filter(\
                                     (col('tpep_pickup_datetime').isNull()) |\
                                     (col('tpep_dropoff_datetime').isNull()) |\
                                     (col('dolocationid').isNull()) |\
                                     (col('pulocationid').isNull()) |\
                                     (col('trip_distance').isNull()) |\
                                     (col('total_amount').isNull()) |\
                                     (col('tip_amount').isNull()) |\
                                     (col('payment_type').isNull())
                                     )

recleanedYELLOW = recleanedYELLOW.filter(\
                                     (col('tpep_pickup_datetime').isNotNull()) &\
                                     (col('tpep_dropoff_datetime').isNotNull()) &\
                                     (col('dolocationid').isNotNull()) &\
                                     (col('pulocationid').isNotNull()) &\
                                     (col('trip_distance').isNotNull()) &\
                                     (col('total_amount').isNotNull()) &\
                                     (col('tip_amount').isNotNull()) &\
                                     (col('payment_type').isNotNull())
                                     )

# Before re-integration, we must remember that we have re-cast a lot of columns as floats and the rest of the data is StringType
everdirtyYELLOW = everdirtyYELLOW.select([col(i).cast("string") for i in everdirtyYELLOW.columns])
recleanedYELLOW = recleanedYELLOW.select([col(i).cast("string") for i in recleanedYELLOW.columns])

# We can then drop the validation columns
recleanedYELLOW = recleanedYELLOW.drop(*['pulocationid_val',\
                                   'dolocationid_val',\
                                   'tpep_pickup_datetime_val',\
                                   'tpep_dropoff_datetime_val',\
                                   'vendorid_val',\
                                   'improvement_surcharge_val',\
                                   'mta_tax_val',\
                                   'extra_val',\
                                   'fare_amount_val',\
                                   'tip_amount_val',\
                                   'tolls_amount_val',\
                                   'total_amount_val',\
                                   'store_and_fwd_flag_val',\
                                   'trip_distance_val',\
                                   'passenger_count_val',\
                                   'trip_type_val',\
                                   'payment_type_val',\
                                   'ratecodeid_val',\
                                   'ehail_fee_val',\
                                   'congestion_surcharge_val',\
                                   'row_val'])

# leaving us with just the filename variable as something which differentiates the schema from our clean data

In [199]:
def YELLOW_combine(cleanedupYELLOWDF,alwaysdirtyYELLOWDF):
    
    reAdded = 0
    repair_filepaths = [xxx.filename for xxx in recleanedYELLOW.select('filename').distinct().collect()]

    for i in range(0,len(repair_filepaths)):
        repair_filepaths[i] = clean_YELLOW_directory+"/all CSV/"+repair_filepaths[i]+"clean"
    
    repair_filepaths.sort()
    
    for i in repair_filepaths:
        tempFilterDF = recleanedYELLOW.filter(col('filename') == i.replace(clean_YELLOW_directory,"").\
                                                                      replace('/all CSV/',"").\
                                                                      replace('csvclean','csv'))
            
        dfi = create_df(i)
        pre_existing = dfi.count()

        if pre_existing == 0:
            dfi = tempFilterDF    
        else:
            dfi = dfi.unionByName(tempFilterDF.drop('filename'))
        #we do not drop_duplicates here, as the operation is incredibly expensive for no benefit

        addedDFI = dfi.count()-pre_existing
        print(str(addedDFI)+" additional records added")
        reAdded = reAdded + addedDFI
        save_df(dfi,ntpath.basename(i), clean_YELLOW_directory)

    print(str(reAdded)+" records added across all files")
    
    if reAdded == 0:
        print("WARNING: REPAIRED RECORDS COULD NOT BE RE-ADDED")
    
    # We have the dirty records saved on a monthly basis already anyway
    # We will therefore save everdirtyYELLOW alone
    # We will then have saved files per month with caught dirty records and one master file with unrecoverable records
    # This will record unrecoverable values, but will not take hours to run
    # We do not coalesce() here as the file is large (and we will not be reading it again outside of spark)
    
    alwaysdirtyYELLOWDF.write.options(header = True).mode("overwrite").\
         csv(os.path.join(dirty_YELLOW_directory, "irreparable_YELLOW_records"))

In [200]:
YELLOW_combine(recleanedYELLOW, everdirtyYELLOW)

opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-01.csvclean
27136 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-02.csvclean
25879 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-03.csvclean
27855 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-04.csvclean
27669 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-05.csvclean
28612 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-06.csvclean
27601 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-07.csvclean
26510 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-08.csvclean
26578 additional records added
opened /home/epb123/output/cleaned YELLOW/all CSV/yellow_tripdata_2009-09.csvclean
27182

In [201]:
# Clean the PySpark outputs from before
YELLOW_cleanCSV_folderPath = cleanup(clean_YELLOW_directory)
with open(v_direc + "YELLOW_cleanCSV_folderPath",'wb') as cleanedYELLOWCSVdirec:
    pickle.dump(YELLOW_cleanCSV_folderPath,cleanedYELLOWCSVdirec)

# YELLOW_dirtyCSV_folderPath = cleanup(dirty_YELLOW_directory)
# with open(v_direc + "YELLOW_dirtyCSV_folderPath",'wb') as dirtyYELLOWCSVdirec:
#     pickle.dump(YELLOW_dirtyCSV_folderPath,dirtyYELLOWCSVdirec)

folder already exists at /home/epb123/output/cleaned YELLOW/all CSV


The run time of our 2.3 implimentation is abysmal, however this seems to be a direct result of the numerous record keeping approaches we keep in order to discover dirty data. The code could be streamlined significantly by simply removing all counts and keeping only the filter items. Interestingly, the native pyspark filtering functions and UDFs seem to perform more similarly than expected - when manipulating our Yellow datasets, they are both slow. 

Finally, had we known filtering for payment types would catch over two million dirty Yellow records, we may not have chosen to use a single dataframe to clean our dirty records.

# Data cleaning complete - onto 2.4